In [3]:
!python -m pip install --user --upgrade pip

  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3


In [4]:
!pip install boto3

  Using cached s3transfer-0.3.4-py2.py3-none-any.whl (69 kB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.17.44
    Uninstalling botocore-1.17.44:
      Successfully uninstalled botocore-1.17.44

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.1.2 requires aiohttp>=3.3.1, which is not installed.
aiobotocore 1.1.2 requires aioitertools>=0.5.1, which is not installed.
aiobotocore 1.1.2 requires wrapt>=1.10.10, which is not installed.
aiobotocore 1.1.2 requires botocore<1.17.45,>=1.17.44, but you have botocore 1.20.8 which is incompatible.


In [5]:
!pip install --user s3fs

  Using cached fsspec-0.8.5-py3-none-any.whl (98 kB)
  Using cached aiohttp-3.7.3-cp38-cp38-win_amd64.whl (634 kB)
  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Using cached aioitertools-0.7.1-py3-none-any.whl (20 kB)
  Using cached botocore-1.17.44-py2.py3-none-any.whl (6.5 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
  Using cached yarl-1.6.3-cp38-cp38-win_amd64.whl (125 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached multidict-5.1.0-cp38-cp38-win_amd64.whl (48 kB)
  Using cached idna-3.1-py3-none-any.whl (58 kB)
  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19553 sha256=909c18e406f0bd396339f7a3c364edab1929959085eae948d0e546e055266a7f
  Stored in directory: c:\users\naga raja paidimarri\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built wrapt


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.17.8 requires botocore<1.21.0,>=1.20.8, but you have botocore 1.17.44 which is incompatible.


In [9]:
!pip install pandas

In [10]:
import s3fs
import pandas as pd
import numpy as np
from attributes import *
import boto3, os
import gc
import pickle
s3 = boto3.resource('s3')
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [5]:
! aws s3 ls

2020-12-21 19:41:16 amplify-myhomely-staging-04110-deployment
2021-01-18 01:50:53 amplify-webapplication-dev-235042-deployment
2020-12-30 17:01:58 cf-templates-zh58d9ea8rbl-us-west-2
2020-12-20 17:50:34 myhomely


In [6]:
flatfile = pd.DataFrame()

In [7]:
# df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'PP.csv'))['POSTCODE']
# df1 = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'ED.csv'))['POSTCODE']
# (df != df1).sum()

In [8]:
def proportionold(df,l = ['IN_HH_5' ,'IN_HH0510' ,'IN_HH1015' ,'IN_HH1520' ,'IN_HH2025' ], d = 'IN_THH'):
    rand = pd.DataFrame()
    rand['rand'] = df[l[0]]
    if len(l) > 1:
        for variable in l[1:]:
            rand['rand'] = rand['rand'] + df[variable]
    return (rand['rand']/df[d]) * 100    

def proportion(df,l = ['IN_HH_5' ,'IN_HH0510' ,'IN_HH1015' ,'IN_HH1520' ,'IN_HH2025' ], d = 'IN_THH'):
    rand = pd.DataFrame()
    rand['rand'] = df[l[0]]
    if len(l) > 1:
        for variable in l[1:]:
            rand['rand'] = rand['rand'] + df[variable]
    return ((rand['rand']/df[d]) * 100).fillna(0)    

## Population

In [9]:
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'PP.csv'))
df.shape

(799313, 94)

In [10]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [11]:
df['temp'] = df['PP_0_4'] + df['PP_5_9'] + df['PP_10_14']
df['PP_0_14'].sum(), df['PP_0_4'].sum(),df['PP_5_9'].sum(),df['PP_10_14'].sum(),df['temp'].sum()

(6069682.833999999,
 1969791.5689999997,
 2039722.4849999999,
 2060169.048,
 6069683.101999998)

In [12]:
df['PP_TOT'].sum()/1000000

38.065518212999976

In [13]:
(df[['PP_0_14' ,'PP_15_64' ,'PP_65_']].sum(axis = 1) - df['PP_TOT']).sum()

0.1299999999753021

In [14]:
(df['PP_MR_CL'] + df['PP_NMRCL']).sum()/1000000

31.99583535

In [15]:
df['PP_POP15_'].sum()/1000000

31.995835286

In [16]:
temp['POSTCODE'] = df['POSTCODE']
temp['PP1'] = df['PP_0_14'] / df['PP_TOT'] * 100
temp['PP2'] = (df['PP_15_19'] + df['PP_20_24'] + df['PP_25_29']) / df['PP_TOT'] * 100
temp['PP3'] = (df['PP_30_34'] + df['PP_35_39'] + df['PP_40_44']) / df['PP_TOT'] * 100
temp['PP4'] = (df['PP_45_49'] + df['PP_50_54'] + df['PP_55_59']) / df['PP_TOT'] * 100
temp['PP5'] = (df['PP_60_64'] + df['PP_65_']) / df['PP_TOT'] * 100

In [17]:
temp['PP6'] = df['PP_PP_MED']

In [18]:
temp['PP7']= df['PP_LMR'] / df['PP_POP15_'] * 100
temp['PP8']= df['PP_NMR'] / df['PP_POP15_'] * 100
temp['PP9']= (df['PP_SEP'] + df['PP_DIVORCE']+ df['PP_WIDOW']+ df['PP_CL_Y'])/ df['PP_POP15_'] * 100

In [19]:
temp = temp.round(2)

In [20]:
flatfile[temp.columns.tolist()] = temp

In [21]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27
...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09


## Education

In [22]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'ED.csv'))
df.shape

(799313, 110)

In [23]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [24]:
(df['ED_15POS']).sum(), (df['ED_15TRC'] + df['ED_15COL'] + df['ED_15UD_B'] + df['ED_15UD']).sum()

(17720153.572000004, 17720153.372)

In [25]:
temp2 = df['ED_15HL'] + df['ED_24HL'] + df['ED_64HL'] + df['ED_65HL']

In [26]:
temp['POSTCODE'] = df['POSTCODE']
temp['ED1'] = (df['ED_15NC'] + df['ED_24NC'] + df['ED_64NC'] + df['ED_65NC']) / temp2 * 100
temp['ED2'] = (df['ED_15HSC']+ df['ED_24HSC'] + df['ED_64HSC'] + df['ED_65HSC']) / temp2 * 100
temp['ED3'] = (df['ED_15TRC']+ df['ED_24TRC'] + df['ED_64TRC'] + df['ED_65TRC']) / temp2 * 100
temp['ED4'] = (df['ED_15COL'] + df['ED_15UD_B'] + df['ED_24COL'] + df['ED_24UD_B'] + df['ED_64COL'] + df['ED_64UD_B'] + df['ED_65COL'] + df['ED_65UD_B']) / temp2 * 100
temp['ED5'] = (df['ED_15BD']+ df['ED_24BD'] + df['ED_64BD'] + df['ED_65BD']) / temp2 * 100
temp['ED6'] = (df['ED_15BDPL'] + df['ED_15MAS'] + df['ED_24BDPL'] + df['ED_24MAS'] + df['ED_64BDPL'] + df['ED_64MAS'] + df['ED_65BDPL'] + df['ED_65MAS']) / temp2 * 100
temp['ED7'] = (df['ED_15MED'] + df['ED_15DOC'] + df['ED_24MED'] + df['ED_24DOC'] + df['ED_64MED'] + df['ED_64DOC'] + df['ED_65MED'] + df['ED_65DOC']) / temp2 * 100

In [27]:
temp.sum(axis = 1).mean()

100.00091663852771

In [28]:
temp = temp.round(2)

In [29]:
flatfile[temp.columns.tolist()] = temp

In [30]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45


## Income

In [31]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'IN2.csv'))
df.shape

(799313, 209)

In [32]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [33]:
temp['POSTCODE'] = df['POSTCODE']
temp['IN1'] = proportion(df,l = ['IN_HH_5' ,'IN_HH0510' ,'IN_HH1015' ,'IN_HH1520' ,'IN_HH2025' ], d = 'IN_THH')
temp['IN2'] = proportion(df,l = ['IN_HH2530' ,'IN_HH3035','IN_HH3540' ,'IN_HH4045' ,'IN_HH4550' ,'IN_HH5060' ], d = 'IN_THH')
temp['IN3'] = proportion(df,l = ['IN_HH6070' ,'IN_HH7080' ,'IN_HH8090' ,'IN_HH90100'], d = 'IN_THH')
temp['IN4'] = proportion(df,l = ['IN_HH100_' ], d = 'IN_THH')

In [34]:
temp.sum(axis = 1).mean()

99.99978355899535

In [35]:
temp['IN5'] = df['IN_MHH']
temp['IN6'] = df["IN_AHH"]

In [36]:
temp

,POSTCODE,IN1,IN2,IN3,IN4,IN5,IN6
0,A0A0B7,17.665615,38.801262,24.500526,19.242902,53852.938,66646.814
1,A0A0C2,14.869110,35.078534,26.178010,24.188482,62993.062,88664.172
2,A0A1A0,9.349166,40.342703,25.990781,24.314845,62638.265,80835.673
3,A0A1B0,5.743809,35.486889,16.579582,42.190064,83602.778,109506.705
4,A0A1C0,10.630832,24.883612,21.127703,43.357496,89877.002,106512.996
...,...,...,...,...,...,...,...
799308,Y1A6W1,0.978070,11.838365,19.209422,67.974144,136407.465,162232.225
799309,Y1A7A1,3.433135,19.760817,26.363224,50.442824,106899.448,112263.815
799310,Y1A7A2,5.216346,25.131156,28.717003,40.935496,92550.382,99121.023
799311,Y1A7A3,3.434519,19.757155,26.360220,50.442324,106784.916,113973.054


In [37]:
temp = temp.round(2)

In [38]:
flatfile[temp.columns.tolist()] = temp

In [39]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00,17.67,38.80,24.50,19.24,53852.94,66646.81
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00,14.87,35.08,26.18,24.19,62993.06,88664.17
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02,9.35,40.34,25.99,24.31,62638.26,80835.67
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00,5.74,35.49,16.58,42.19,83602.78,109506.70
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00,10.63,24.88,21.13,43.36,89877.00,106513.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10,0.98,11.84,19.21,67.97,136407.46,162232.22
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106899.45,112263.82
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54,5.22,25.13,28.72,40.94,92550.38,99121.02
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106784.92,113973.05


## Employement

In [40]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'EM.csv'))
df.shape

(799313, 98)

In [41]:
df.head()

,PostCode,EM_TOT,EM_NA,EM_ALL,EM_PEM,EM_PSM,EM_FTOT,EM_FNA,EM_FALL,EM_FPEM,EM_FPSM,EM_MTOT,EM_MNA,EM_MALL,EM_MPEM,EM_MPSM,EM_PLTOT,EM_PLHOM,EM_PLOCA,EM_PLNOF,EM_PLU,EM_TRTOT,EM_TRCD,EM_TRCP,EM_TRPB,EM_TRWK,EM_TRBK,EM_TROT,EM_FPLTOT,EM_FPLHOM,EM_FPLOCA,EM_FPLNOF,EM_FPLU,EM_FTRTOT,EM_FTRCD,EM_FTRCP,EM_FTRPB,EM_FTRWK,EM_FTRBK,EM_FTROT,EM_MPLTOT,EM_MPLHOM,EM_MPLOCA,EM_MPLNOF,EM_MPLU,EM_MTRTOT,EM_MTRCD,EM_MTRCP,EM_MTRPB,EM_MTRWK,EM_MTRBK,EM_MTROT,EM_CMTOT,EM_CM_15,EM_CM15_29,EM_CM30_44,EM_CM45_59,EM_CM60_,EM_TMTOT,EM_TM5_6,EM_TM6_7,EM_TM7_8,EM_TM8_9,EM_TM9_12,EM_TM12_,EM_LGTOT,EM_LGSIN,EM_LGOFFIC,EM_LGEN,EM_LGFR,EM_LGNOF,EM_LGCRE,EM_LGINU,EM_LGGER,EM_LGPUN,EM_LGPOR,EM_LGSP,EM_LGKOR,EM_LGCAN,EM_LGMAN,EM_LGCH,EM_LGOT,EM_LGMUL,EM_LGENFR,EM_LGENNO,EM_LGFRNO,EM_LGEFNO,EM_LGTOTA,EM_LGNONE,EM_LGENA,EM_LGFRA,EM_LGNOFA,EM_ABOR,EM_NOAB,EM_LGENFRA,EM_LGENNOA,EM_LGFRNOA,EM_LGEFNOA
0,A0A0B7,0.532,0.062,0.470,0.470,0.000,0.201,0.000,0.201,0.201,0.000,0.331,0.062,0.269,0.269,0.000,0.409,0.000,0.000,0.061,0.348,0.399,0.255,0.050,0.000,0.056,0.000,0.039,0.190,0.000,0.000,0.000,0.190,0.186,0.107,0.050,0.000,0.029,0.000,0.000,0.218,0.000,0.000,0.061,0.158,0.213,0.148,0.000,0.000,0.026,0.000,0.039,0.399,0.209,0.100,0.000,0.089,0.000,0.399,0.051,0.098,0.098,0.048,0.051,0.052,0.592,0.592,0.592,0.592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.592,0.592,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A0A0C2,0.878,0.000,0.878,0.738,0.140,0.356,0.000,0.356,0.356,0.000,0.522,0.000,0.522,0.382,0.140,0.815,0.151,0.000,0.307,0.356,0.739,0.655,0.051,0.000,0.000,0.000,0.033,0.329,0.057,0.000,0.082,0.190,0.337,0.304,0.000,0.000,0.000,0.000,0.033,0.485,0.094,0.000,0.225,0.166,0.402,0.351,0.051,0.000,0.000,0.000,0.000,0.739,0.173,0.358,0.048,0.000,0.159,0.739,0.052,0.194,0.107,0.077,0.055,0.254,1.032,1.032,1.032,1.032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.032,1.032,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A0A1A0,21.716,7.636,14.080,13.496,0.584,9.600,3.430,6.170,5.855,0.315,12.116,4.206,7.909,7.640,0.269,15.073,0.734,0.576,3.959,9.804,12.379,8.650,0.668,0.374,0.164,0.029,2.494,3.122,0.706,0.183,0.235,1.998,4.049,2.630,0.641,0.309,0.128,0.018,0.324,11.952,0.028,0.393,3.725,7.806,8.330,6.021,0.027,0.065,0.036,0.011,2.171,12.379,3.075,3.819,3.687,0.772,1.027,12.379,2.172,2.399,2.488,0.644,0.369,4.309,26.179,26.179,26.179,26.179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.179,26.179,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A0A1B0,219.271,0.030,219.241,196.429,22.811,100.677,0.016,100.660,93.973,6.687,118.594,0.014,118.581,102.456,16.124,190.860,7.806,0.000,38.431,144.623,169.675,133.958,16.826,6.057,6.685,0.000,6.149,89.102,7.806,0.000,6.888,74.409,81.297,62.996,11.616,0.000,6.685,0.000,0.000,101.758,0.000,0.000,31.543,70.215,88.378,70.962,5.211,6.057,0.000,0.000,6.149,169.675,20.658,10.436,45.724,51.970,40.887,169.675,44.231,33.396,44.480,18.417,10.879,18.272,253.237,253.237,253.237,253.237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,253.237,245.479,0.0,7.759,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A0A1C0,831.419,23.641,807.778,762.349,45.429,369.390,0.000,369.390,349.007,20.383,462.029,23.641,438.388,413.342,25.046,726.270,27.406,11.634,113.927,573.303,681.093,612.378,40.986,0.000,8.037,0.000,19.692,343.814,22.146,0.000,9.981,311.687,311.214,290.304,11.280,0.000,4.529,0.000,5.101,382.456,5.260,11.634,103.946,261.616,369.879,322.074,29.705,0.000,3.508,0.000,14.591,681.093,155.764,144.760,267.090,52.055,61.424,681.093,47.352,126.198,230.323,139.420,48.237,89.563,958.204,958.204,958.204,958.204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,958.204,949.532,0.0,8.672,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [43]:
temp['POSTCODE'] = df['PostCode']
div = 'EM_TRTOT'
temp['EM1'] = proportion(df,l = ['EM_TRCD', 'EM_TRCP'], d = div)
temp['EM2'] = proportion(df,l = ['EM_TRPB'], d = div)
temp['EM3'] = proportion(df,l = ['EM_TRWK'], d = div)
temp['EM4'] = proportion(df,l = ['EM_TRBK'], d = div)
temp['EM5'] = proportion(df,l = ['EM_TROT'], d = div)

In [44]:
div = 'EM_CMTOT'
temp['EM6'] = proportion(df,l = ['EM_CM_15','EM_CM15_29'], d = div)
temp['EM7'] = proportion(df,l = ['EM_CM30_44' ,'EM_CM45_59'], d = div)
temp['EM8'] = proportion(df,l = ['EM_CM60_'], d = div)

In [45]:
temp.sum(axis = 1).mean()

199.99807277064866

In [46]:
temp

,POSTCODE,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8
0,A0A0B7,76.441103,0.000000,14.035088,0.000000,9.774436,77.443609,22.305764,0.000000
1,A0A0C2,95.534506,0.000000,0.000000,0.000000,4.465494,71.853857,6.495264,21.515562
2,A0A1A0,75.272639,3.021246,1.324824,0.234268,20.147023,55.691090,36.020680,8.296308
3,A0A1B0,88.866362,3.569766,3.939885,0.000000,3.623987,18.325623,57.577133,24.097245
4,A0A1C0,95.928750,0.000000,1.180015,0.000000,2.891235,44.123783,46.857771,9.018445
...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,93.677224,2.225249,1.500125,1.200867,1.396536,94.183660,4.795795,1.022464
799309,Y1A7A1,78.287028,11.202104,7.702984,2.807723,0.000000,94.478316,5.521684,0.000000
799310,Y1A7A2,76.623481,5.879999,15.473547,2.023259,0.000000,93.675850,6.324436,0.000000
799311,Y1A7A3,78.284826,11.201955,7.704954,2.808265,0.000000,94.476783,5.523217,0.000000


In [47]:
temp = temp.round(2)

In [48]:
flatfile[temp.columns.tolist()] = temp

In [49]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00,17.67,38.80,24.50,19.24,53852.94,66646.81,76.44,0.00,14.04,0.00,9.77,77.44,22.31,0.00
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00,14.87,35.08,26.18,24.19,62993.06,88664.17,95.53,0.00,0.00,0.00,4.47,71.85,6.50,21.52
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02,9.35,40.34,25.99,24.31,62638.26,80835.67,75.27,3.02,1.32,0.23,20.15,55.69,36.02,8.30
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00,5.74,35.49,16.58,42.19,83602.78,109506.70,88.87,3.57,3.94,0.00,3.62,18.33,57.58,24.10
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00,10.63,24.88,21.13,43.36,89877.00,106513.00,95.93,0.00,1.18,0.00,2.89,44.12,46.86,9.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10,0.98,11.84,19.21,67.97,136407.46,162232.22,93.68,2.23,1.50,1.20,1.40,94.18,4.80,1.02
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106899.45,112263.82,78.29,11.20,7.70,2.81,0.00,94.48,5.52,0.00
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54,5.22,25.13,28.72,40.94,92550.38,99121.02,76.62,5.88,15.47,2.02,0.00,93.68,6.32,0.00
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106784.92,113973.05,78.28,11.20,7.70,2.81,0.00,94.48,5.52,0.00


## Minority

In [50]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'MN.csv'))
df.shape

(799313, 16)

In [51]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [52]:
df.head()

,POSTCODE,MN_TOT,MN_VIS,MN_SASIAN,MN_CHINESE,MN_BLACK,MN_FILIPIN,MN_LAMER,MN_ARAB_,MN_SEASIA_,MN_WASIAN,MN_KOREAN,MN_JAPANE,MN_NIE,MN_MULT,MN_OT
0,A0A0B7,2.021,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.021
1,A0A0C2,2.010,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.010
2,A0A1A0,71.259,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.259
3,A0A1B0,596.075,0.00,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,596.075
4,A0A1C0,1552.877,34.15,23.395,10.755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1518.727


In [53]:
temp['POSTCODE'] = df['POSTCODE']
div = 'MN_TOT'
temp['MN1'] = proportion(df,l = ['MN_VIS'], d = div)
temp['MN2'] = proportion(df,l = ['MN_OT'], d = div)

In [54]:
div = 'MN_VIS'
temp['MN3'] = proportion(df,l = ['MN_SASIAN'], d = div)
temp['MN4'] = proportion(df,l = ['MN_CHINESE'], d = div)
temp['MN5'] = proportion(df,l = ['MN_BLACK'], d = div)
temp['MN6'] = proportion(df,l = ['MN_LAMER'], d = div)
temp['MN7'] = proportion(df,l = ['MN_ARAB_','MN_WASIAN'], d = div)
temp['MN8'] = proportion(df,l = ['MN_FILIPIN' ,'MN_SEASIA_' ,'MN_KOREAN' ,'MN_JAPANE'], d = div)
temp['MN9'] = proportion(df,l = ['MN_NIE' ,'MN_MULT'], d = div)

In [55]:
temp[temp['MN1'] != 0].sum(axis = 1).mean()

199.99181277246458

In [56]:
temp

,POSTCODE,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9
0,A0A0B7,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,A0A0C2,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,A0A1A0,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,A0A1B0,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,A0A1C0,2.199144,97.800856,68.506589,31.493411,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,11.857801,88.142199,28.417916,0.000000,4.569261,3.630125,0.0,63.382698,0.0
799309,Y1A7A1,10.847859,89.152141,17.330994,0.000000,0.000000,0.000000,0.0,82.669006,0.0
799310,Y1A7A2,6.566224,93.433625,18.208510,0.000000,0.000000,0.000000,0.0,81.791490,0.0
799311,Y1A7A3,10.848749,89.151251,17.329020,0.000000,0.000000,0.000000,0.0,82.670980,0.0


In [57]:
temp = temp.round(2)

In [58]:
flatfile[temp.columns.tolist()] = temp

In [59]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00,17.67,38.80,24.50,19.24,53852.94,66646.81,76.44,0.00,14.04,0.00,9.77,77.44,22.31,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00,14.87,35.08,26.18,24.19,62993.06,88664.17,95.53,0.00,0.00,0.00,4.47,71.85,6.50,21.52,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02,9.35,40.34,25.99,24.31,62638.26,80835.67,75.27,3.02,1.32,0.23,20.15,55.69,36.02,8.30,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00,5.74,35.49,16.58,42.19,83602.78,109506.70,88.87,3.57,3.94,0.00,3.62,18.33,57.58,24.10,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00,10.63,24.88,21.13,43.36,89877.00,106513.00,95.93,0.00,1.18,0.00,2.89,44.12,46.86,9.02,2.20,97.80,68.51,31.49,0.00,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10,0.98,11.84,19.21,67.97,136407.46,162232.22,93.68,2.23,1.50,1.20,1.40,94.18,4.80,1.02,11.86,88.14,28.42,0.00,4.57,3.63,0.0,63.38,0.0
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106899.45,112263.82,78.29,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54,5.22,25.13,28.72,40.94,92550.38,99121.02,76.62,5.88,15.47,2.02,0.00,93.68,6.32,0.00,6.57,93.43,18.21,0.00,0.00,0.00,0.0,81.79,0.0
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106784.92,113973.05,78.28,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0


## Occupation

In [60]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'OC.csv'))
df.shape

(799313, 40)

In [61]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [62]:
df.head(1)

,PostCode,OC_TOT,OC_NA,OC_ALL,OC_MG,OC_BS,OC_SC,OC_HL,OC_SSC,OC_ART,OC_SAL,OC_TR,OC_UN,OC_UNPMU,OC_FTOT,OC_FNA,OC_FALL,OC_FMG,OC_FBS,OC_FSC,OC_FHL,OC_FSSC,OC_FART,OC_FSAL,OC_FTR,OC_FUN,OC_FUNPMU,OC_MTOT,OC_MNA,OC_MALL,OC_MMG,OC_MBS,OC_MSC,OC_MHL,OC_MSSC,OC_MART,OC_MSAL,OC_MTR,OC_MUN,OC_MUNPMU
0,A0A0B7,0.532,0.062,0.47,0.0,0.0,0.063,0.0,0.095,0.0,0.077,0.107,0.058,0.071,0.201,0.0,0.201,0.0,0.0,0.01,0.0,0.095,0.0,0.038,0.0,0.058,0.0,0.331,0.062,0.269,0.0,0.0,0.052,0.0,0.0,0.0,0.039,0.107,0.0,0.071


In [63]:
temp['POSTCODE'] = df['PostCode']
div = 'OC_ALL'
temp['OC1'] = proportion(df,l = ['OC_MG'], d = div)
temp['OC2'] = proportion(df,l = ['OC_BS'], d = div)
temp['OC3'] = proportion(df,l = ['OC_SC'], d = div)
temp['OC4'] = proportion(df,l = ['OC_HL'], d = div)
temp['OC5'] = proportion(df,l = ['OC_SSC'], d = div)
temp['OC6'] = proportion(df,l = ['OC_ART'], d = div)
temp['OC7'] = proportion(df,l = ['OC_SAL'], d = div)
temp['OC8'] = proportion(df,l = ['OC_TR'], d = div)
temp['OC9'] = proportion(df,l = ['OC_UN'], d = div)
temp['OC10'] = proportion(df,l = ['OC_UNPMU'], d = div)

In [64]:
temp.sum(axis = 1).mean()

99.99990591634294

In [65]:
temp

,POSTCODE,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10
0,A0A0B7,0.000000,0.000000,13.404255,0.000000,20.212766,0.000000,16.382979,22.765957,12.340426,15.106383
1,A0A0C2,0.000000,0.000000,0.000000,4.441913,13.211845,0.000000,8.883827,18.564920,38.382688,16.400911
2,A0A1A0,4.552557,11.903409,1.669034,5.994318,8.742898,1.356534,11.725852,0.830966,29.332386,23.884943
3,A0A1B0,5.445149,19.774130,7.162438,3.638462,12.678286,2.728960,14.700261,27.528610,3.638918,2.704330
4,A0A1C0,2.494621,19.765208,11.074701,12.501950,9.957067,1.794429,12.122266,17.866543,6.847797,5.575542
...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,15.113680,15.422641,9.036221,10.559221,21.022118,3.442197,13.725169,10.419279,1.257656,0.000000
799309,Y1A7A1,5.582040,14.373572,8.810825,10.070587,26.735560,3.356912,17.178804,12.228413,0.000000,1.663434
799310,Y1A7A2,5.581858,14.373491,8.810728,10.070697,26.735487,3.356899,17.178863,12.228582,0.000000,1.663395
799311,Y1A7A3,5.582455,14.375229,8.809049,10.070391,26.736380,3.356797,17.178663,12.226879,0.000000,1.664158


In [66]:
temp = temp.round(2)

In [67]:
flatfile[temp.columns.tolist()] = temp

In [68]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00,17.67,38.80,24.50,19.24,53852.94,66646.81,76.44,0.00,14.04,0.00,9.77,77.44,22.31,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,13.40,0.00,20.21,0.00,16.38,22.77,12.34,15.11
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00,14.87,35.08,26.18,24.19,62993.06,88664.17,95.53,0.00,0.00,0.00,4.47,71.85,6.50,21.52,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,4.44,13.21,0.00,8.88,18.56,38.38,16.40
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02,9.35,40.34,25.99,24.31,62638.26,80835.67,75.27,3.02,1.32,0.23,20.15,55.69,36.02,8.30,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,4.55,11.90,1.67,5.99,8.74,1.36,11.73,0.83,29.33,23.88
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00,5.74,35.49,16.58,42.19,83602.78,109506.70,88.87,3.57,3.94,0.00,3.62,18.33,57.58,24.10,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,5.45,19.77,7.16,3.64,12.68,2.73,14.70,27.53,3.64,2.70
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00,10.63,24.88,21.13,43.36,89877.00,106513.00,95.93,0.00,1.18,0.00,2.89,44.12,46.86,9.02,2.20,97.80,68.51,31.49,0.00,0.00,0.0,0.00,0.0,2.49,19.77,11.07,12.50,9.96,1.79,12.12,17.87,6.85,5.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10,0.98,11.84,19.21,67.97,136407.46,162232.22,93.68,2.23,1.50,1.20,1.40,94.18,4.80,1.02,11.86,88.14,28.42,0.00,4.57,3.63,0.0,63.38,0.0,15.11,15.42,9.04,10.56,21.02,3.44,13.73,10.42,1.26,0.00
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106899.45,112263.82,78.29,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0,5.58,14.37,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54,5.22,25.13,28.72,40.94,92550.38,99121.02,76.62,5.88,15.47,2.02,0.00,93.68,6.32,0.00,6.57,93.43,18.21,0.00,0.00,0.00,0.0,81.79,0.0,5.58,14.37,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106784.92,113973.05,78.28,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0,5.58,14.38,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66


## Dwelling

In [69]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'DW.csv'))
df.shape

(799313, 69)

In [70]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

DW_VALSTD    0.00025
dtype: float64

In [71]:
df.head(1)

,postcode,DW_TOT,DW_SINGLE,DW_APT5_,DW_OT,DW_SEMI,DW_ROW,DW_DUP,DW_APT_5,DW_OT_SING,DW_MOVA,DW_TOTDWL5,DW_OWNED,DW_RENTED,DW_BANDH,DW_TOTDWL6,DW_PTCON,DW_NPTCON,DW_TOTDWL4,DW_BRM0,DW_BRM1,DW_BRM2,DW_BRM3,DW_BRM4_,DW_TOTDWL3,DW_RM_4,DW_RM5,DW_RM6,DW_RM7,DW_RM8_,DW_ARMSPDW,DW_TOTDWL9,DW_PPR_1,DW_PPR2_,DW_TOTDWLA,DW_SUIT,DW_NSUIT,DW_TOTDWL2,DW_CON_60,DW_CON6180,DW_CON8190,DW_CON9100,DW_CON0105,DW_CON0610,DW_CON1116,DW_CON17_,DW_TOTDWLG,DW_REGMIN,DW_MAJORRE,DW_TOTDWL7,DW_MT1,DW_MT2,DW_MT3_,DW_TOTDWL8,DW_MT_25,DW_MT2534,DW_MT3544,DW_MT4554,DW_MT5564,DW_MT6574,DW_MT7584,DW_MT85_,DW_MCOST,DW_ACOST,DW_MVALUE,DW_AVALUE,DW_MINVAL,DW_MAXVAL,DW_VALSTD
0,A0A0B7,0.951,0.951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.951,0.951,0.0,0.0,0.951,0.0,0.951,0.951,0.0,0.0,0.329,0.451,0.17,0.951,0.097,0.182,0.185,0.32,0.167,6.166,0.951,0.951,0.0,0.951,0.951,0.0,0.951,0.283,0.391,0.188,0.09,0.0,0.0,0.0,0.0,0.951,0.819,0.132,0.951,0.667,0.284,0.0,0.951,0.0,0.0,0.128,0.087,0.269,0.321,0.145,0.0,462.649,644.581,163130.701,163130.701,163130.701,163130.701,0.0


In [72]:
temp['POSTCODE'] = df['postcode']
div = 'DW_TOT'
temp['DW1'] = proportion(df,l = ['DW_SINGLE'], d = div)
temp['DW2'] = proportion(df,l = ['DW_SEMI'], d = div)
temp['DW3'] = proportion(df,l = ['DW_ROW'], d = div)
temp['DW4'] = proportion(df,l = ['DW_APT5_','DW_DUP','DW_APT_5'], d = div)
temp['DW5'] = proportion(df,l = ['DW_OT_SING','DW_MOVA'], d = div)

In [73]:
div = 'DW_TOTDWL5'
temp['DW6'] = proportion(df,l = ['DW_OWNED'], d = div)
temp['DW7'] = proportion(df,l = ['DW_RENTED'], d = div)
temp['DW8'] = proportion(df,l = ['DW_BANDH'], d = div)

In [74]:
temp.sum(axis = 1).mean()

199.99999135479715

In [75]:
temp

,POSTCODE,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8
0,A0A0B7,100.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.0
1,A0A0C2,100.000000,0.000000,0.000000,0.000000,0.000000,95.078534,4.921466,0.0
2,A0A1A0,99.403778,0.170349,0.248008,0.177865,0.000000,89.017486,10.985019,0.0
3,A0A1B0,94.811722,0.000000,3.432939,1.754995,0.000000,79.661384,20.338272,0.0
4,A0A1C0,96.408993,0.631172,2.885432,0.074403,0.000000,90.547451,9.452549,0.0
...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,74.800325,18.776229,0.000000,6.423447,0.000000,83.112224,16.884391,0.0
799309,Y1A7A1,45.510788,42.830953,9.155097,2.502955,0.000000,70.618996,29.381004,0.0
799310,Y1A7A2,57.984338,34.207473,5.255606,1.113520,1.438716,66.669099,33.330901,0.0
799311,Y1A7A3,45.510263,42.833189,9.152934,2.503614,0.000000,70.615785,29.378433,0.0


In [76]:
temp = temp.round(2)

In [77]:
flatfile[temp.columns.tolist()] = temp

In [78]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00,17.67,38.80,24.50,19.24,53852.94,66646.81,76.44,0.00,14.04,0.00,9.77,77.44,22.31,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,13.40,0.00,20.21,0.00,16.38,22.77,12.34,15.11,100.00,0.00,0.00,0.00,0.00,100.00,0.00,0.0
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00,14.87,35.08,26.18,24.19,62993.06,88664.17,95.53,0.00,0.00,0.00,4.47,71.85,6.50,21.52,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,4.44,13.21,0.00,8.88,18.56,38.38,16.40,100.00,0.00,0.00,0.00,0.00,95.08,4.92,0.0
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02,9.35,40.34,25.99,24.31,62638.26,80835.67,75.27,3.02,1.32,0.23,20.15,55.69,36.02,8.30,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,4.55,11.90,1.67,5.99,8.74,1.36,11.73,0.83,29.33,23.88,99.40,0.17,0.25,0.18,0.00,89.02,10.99,0.0
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00,5.74,35.49,16.58,42.19,83602.78,109506.70,88.87,3.57,3.94,0.00,3.62,18.33,57.58,24.10,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,5.45,19.77,7.16,3.64,12.68,2.73,14.70,27.53,3.64,2.70,94.81,0.00,3.43,1.75,0.00,79.66,20.34,0.0
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00,10.63,24.88,21.13,43.36,89877.00,106513.00,95.93,0.00,1.18,0.00,2.89,44.12,46.86,9.02,2.20,97.80,68.51,31.49,0.00,0.00,0.0,0.00,0.0,2.49,19.77,11.07,12.50,9.96,1.79,12.12,17.87,6.85,5.58,96.41,0.63,2.89,0.07,0.00,90.55,9.45,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10,0.98,11.84,19.21,67.97,136407.46,162232.22,93.68,2.23,1.50,1.20,1.40,94.18,4.80,1.02,11.86,88.14,28.42,0.00,4.57,3.63,0.0,63.38,0.0,15.11,15.42,9.04,10.56,21.02,3.44,13.73,10.42,1.26,0.00,74.80,18.78,0.00,6.42,0.00,83.11,16.88,0.0
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106899.45,112263.82,78.29,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0,5.58,14.37,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66,45.51,42.83,9.16,2.50,0.00,70.62,29.38,0.0
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54,5.22,25.13,28.72,40.94,92550.38,99121.02,76.62,5.88,15.47,2.02,0.00,93.68,6.32,0.00,6.57,93.43,18.21,0.00,0.00,0.00,0.0,81.79,0.0,5.58,14.37,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66,57.98,34.21,5.26,1.11,1.44,66.67,33.33,0.0
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106784.92,113973.05,78.28,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0,5.58,14.38,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66,45.51,42.83,9.15,2.50,0.00,70.62,29.38,0.0


## Household

In [79]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'HH.csv'))
df.shape

(799313, 29)

In [80]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

Series([], dtype: float64)

In [81]:
df.head(1)

,POSTCODE,HH_TOT,HH_1P,HH_2P,HH_3P,HH_4P,HH_5P,HH_NUMPINH,HH_AVGPINH,HH_TOTHH,HH_ONEFMHH,HH_CP_NC,HH_CP_CH,HH_MULFMHH,HH_NOFMHH,HH_NOFMHH1,HH_NOFMHH2,HH_TOT_A,HH_REOW_30,HH_REOW30,HH_REOW99,HH_OWN,HH_PMORT,HH_POWN30,HH_RENT,HH_PRENT,HH_PRENT30,HH_MRENT,HH_ARENT
0,A0A0B7,0.951,0.36,0.317,0.161,0.095,0.018,2.001,2.104,0.951,0.662,0.333,0.329,0.006,0.283,0.247,0.036,0.951,0.951,0.0,0.0,0.951,29.454,16.999,0.0,0.0,0.0,0.0,0.0


In [82]:
df['HH_ONEFMHH'] = df['HH_ONEFMHH'].replace(0,1)

In [83]:
(df['HH_TOT_A'] - (df['HH_OWN'] + df['HH_RENT'])).sum()

162.91700000000023

In [84]:
temp['POSTCODE'] = df['POSTCODE']
div = 'HH_ONEFMHH'
temp['HH1'] = proportion(df,l = ['HH_CP_NC'], d = div)
temp['HH2'] = proportion(df,l = ['HH_CP_CH'], d = div)

In [85]:
temp.sum(axis = 1).mean()

99.99722257973288

In [86]:
temp

,POSTCODE,HH1,HH2
0,A0A0B7,50.302115,49.697885
1,A0A0C2,48.311307,51.688693
2,A0A1A0,67.313528,32.686472
3,A0A1B0,48.556070,51.443930
4,A0A1C0,38.728013,61.271987
...,...,...,...
799308,Y1A6W1,28.024284,71.975716
799309,Y1A7A1,35.454578,64.545422
799310,Y1A7A2,40.058940,59.941060
799311,Y1A7A3,35.451531,64.548469


In [87]:
temp = temp.round(2)

In [88]:
flatfile[temp.columns.tolist()] = temp

In [89]:
flatfile

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,HH1,HH2
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.00,17.67,38.80,24.50,19.24,53852.94,66646.81,76.44,0.00,14.04,0.00,9.77,77.44,22.31,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,13.40,0.00,20.21,0.00,16.38,22.77,12.34,15.11,100.00,0.00,0.00,0.00,0.00,100.00,0.00,0.0,50.30,49.70
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.00,14.87,35.08,26.18,24.19,62993.06,88664.17,95.53,0.00,0.00,0.00,4.47,71.85,6.50,21.52,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,4.44,13.21,0.00,8.88,18.56,38.38,16.40,100.00,0.00,0.00,0.00,0.00,95.08,4.92,0.0,48.31,51.69
2,A0A1A0,6.00,9.68,6.79,20.85,56.67,62.74,57.90,19.49,22.61,28.42,35.78,9.36,23.77,0.92,1.72,0.02,9.35,40.34,25.99,24.31,62638.26,80835.67,75.27,3.02,1.32,0.23,20.15,55.69,36.02,8.30,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,4.55,11.90,1.67,5.99,8.74,1.36,11.73,0.83,29.33,23.88,99.40,0.17,0.25,0.18,0.00,89.02,10.99,0.0,67.31,32.69
3,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.00,5.74,35.49,16.58,42.19,83602.78,109506.70,88.87,3.57,3.94,0.00,3.62,18.33,57.58,24.10,0.00,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,5.45,19.77,7.16,3.64,12.68,2.73,14.70,27.53,3.64,2.70,94.81,0.00,3.43,1.75,0.00,79.66,20.34,0.0,48.56,51.44
4,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.00,10.63,24.88,21.13,43.36,89877.00,106513.00,95.93,0.00,1.18,0.00,2.89,44.12,46.86,9.02,2.20,97.80,68.51,31.49,0.00,0.00,0.0,0.00,0.0,2.49,19.77,11.07,12.50,9.96,1.79,12.12,17.87,6.85,5.58,96.41,0.63,2.89,0.07,0.00,90.55,9.45,0.0,38.73,61.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799308,Y1A6W1,22.82,21.30,22.61,23.31,9.96,35.00,56.02,24.26,19.71,12.68,24.70,8.66,26.04,19.47,6.35,2.10,0.98,11.84,19.21,67.97,136407.46,162232.22,93.68,2.23,1.50,1.20,1.40,94.18,4.80,1.02,11.86,88.14,28.42,0.00,4.57,3.63,0.0,63.38,0.0,15.11,15.42,9.04,10.56,21.02,3.44,13.73,10.42,1.26,0.00,74.80,18.78,0.00,6.42,0.00,83.11,16.88,0.0,28.02,71.98
799309,Y1A7A1,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106899.45,112263.82,78.29,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0,5.58,14.37,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66,45.51,42.83,9.16,2.50,0.00,70.62,29.38,0.0,35.45,64.55
799310,Y1A7A2,17.76,15.62,24.39,22.03,20.21,39.97,35.77,31.14,33.09,13.64,16.13,9.85,23.73,23.23,10.88,2.54,5.22,25.13,28.72,40.94,92550.38,99121.02,76.62,5.88,15.47,2.02,0.00,93.68,6.32,0.00,6.57,93.43,18.21,0.00,0.00,0.00,0.0,81.79,0.0,5.58,14.37,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66,57.98,34.21,5.26,1.11,1.44,66.67,33.33,0.0,40.06,59.94
799311,Y1A7A3,17.76,15.62,24.38,22.03,20.21,39.97,35.77,31.14,33.09,10.64,17.15,8.30,22.40,25.45,13.61,2.45,3.43,19.76,26.36,50.44,106784.92,113973.05,78.28,11.20,7.70,2.81,0.00,94.48,5.52,0.00,10.85,89.15,17.33,0.00,0.00,0.00,0.0,82.67,0.0,5.58,14.38,8.81,10.07,26.74,3.36,17.18,12.23,0.00,1.66,45.51,42.83,9.15,2.50,0.00,70.62,29.38,0.0,35.45,64.55


In [90]:
flatfile.isna().sum()

POSTCODE    0
PP1         0
PP2         0
PP3         0
PP4         0
PP5         0
PP6         0
PP7         0
PP8         0
PP9         0
ED1         0
ED2         0
ED3         0
ED4         0
ED5         0
ED6         0
ED7         0
IN1         0
IN2         0
IN3         0
IN4         0
IN5         0
IN6         0
EM1         0
EM2         0
EM3         0
EM4         0
EM5         0
EM6         0
EM7         0
EM8         0
MN1         0
MN2         0
MN3         0
MN4         0
MN5         0
MN6         0
MN7         0
MN8         0
MN9         0
OC1         0
OC2         0
OC3         0
OC4         0
OC5         0
OC6         0
OC7         0
OC8         0
OC9         0
OC10        0
DW1         0
DW2         0
DW3         0
DW4         0
DW5         0
DW6         0
DW7         0
DW8         0
HH1         0
HH2         0
dtype: int64

In [91]:
# cccccc

# RPS

In [92]:
del df
temp = pd.DataFrame()
df = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/rps', '202010.csv'), encoding = 'latin-1')
df.shape

(1112905, 36)

In [93]:
temp1 = pd.DataFrame()
temp2 = pd.DataFrame()
temp3 = pd.DataFrame()

In [94]:
percent_missing = df.isna().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
percent_missing[percent_missing>0]

CMA_Name             8.546192
Avg_BED              0.005661
StdDev_BED           0.005661
Avg_LOTSIZE         12.081265
StdDev_LOTSIZE      12.081265
Avg_YEARBUILT        0.003864
StdDev_YEARBUILT     0.003864
Avg_PARKINGCODE      3.788464
dtype: float64

In [95]:
df.head()

,FSALDU,PRCDDA,DateYYYYMM,CMA_Name,PropertyStyle,Min_LowerBound,LowerBound,ValueEstimate,UpperBound,Max_UpperBound,Avg_BED,StdDev_BED,Avg_BATH,StdDev_BATH,Avg_LIVING_AREA,StdDev_LIVING_AREA,Avg_LOTSIZE,StdDev_LOTSIZE,Avg_YEARBUILT,StdDev_YEARBUILT,Avg_PARKINGCODE,Mode_PARKINGTYPE,Mode_BASEMENT,Prox_Accommodation_Score,Prox_Airports_Score,Prox_Cinemas_Score,Prox_Education_Score,Prox_Financial_Score,Prox_Health_Score,Prox_Water_Score,Prox_Golf_Score,Prox_ShoppingCentre_Score,Prox_Stadium_Score,Prox_Commerical_Score,Homogeneity,Resilience_Score
0,A1E2E2,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821,0.547372,0.621751
1,A1E3Z8,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821,0.547372,0.621751
2,A1E4A1,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821,0.547372,0.621751
3,A1E4A5,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821,0.547372,0.621751
4,A1E4A6,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821,0.547372,0.621751


In [96]:
df.groupby('FSALDU').PropertyStyle.nunique().sort_values(ascending = False).head(4)

FSALDU
H3X4B3    4
T2E8M6    4
G6W2M4    4
M6P3P2    4
Name: PropertyStyle, dtype: int64

In [97]:
df.PropertyStyle.unique()

array(['01_SFD', '04_ROW', '02_SD', '03_CONDO'], dtype=object)

In [98]:
# property style SFD

In [99]:
df1 = df[df['PropertyStyle'] == '01_SFD']

In [100]:
temp['POSTCODE'] = df1['FSALDU']
temp['RPS1'] = df1['LowerBound']
temp['RPS2'] = df1['ValueEstimate']
temp['RPS3'] = df1['UpperBound']
temp['RPS4'] = df1['Avg_BED']
temp['RPS5'] = df1['Avg_BATH']
# temp['RPS6'] = df1['Avg_LIVING_AREA']
# temp['RPS7'] = df1['Avg_LOTSIZE']
# temp['RPS8'] = df1['Avg_YEARBUILT']
# temp['RPS9'] = df1['Avg_PARKINGCODE']
# temp['RPS10'] = df1['Homogeneity']
# temp['RPS11'] = df1['Resilience_Score']

In [101]:
temp.isna().sum() * 100 / len(temp)

POSTCODE    0.0
RPS1        0.0
RPS2        0.0
RPS3        0.0
RPS4        0.0
RPS5        0.0
dtype: float64

In [102]:
df1 = df[df['PropertyStyle'] == '02_SD']

In [103]:
temp1['POSTCODE'] = df1['FSALDU']
temp1['RPS12'] = df1['LowerBound']
temp1['RPS13'] = df1['ValueEstimate']
temp1['RPS14'] = df1['UpperBound']
temp1['RPS15'] = df1['Avg_BED']
temp1['RPS16'] = df1['Avg_BATH']
# temp1['RPS17'] = df1['Avg_LIVING_AREA']
# temp1['RPS18'] = df1['Avg_LOTSIZE']
# temp1['RPS19'] = df1['Avg_YEARBUILT']
# temp1['RPS20'] = df1['Avg_PARKINGCODE']
# temp1['RPS21'] = df1['Homogeneity']
# temp1['RPS22'] = df1['Resilience_Score']

In [104]:
temp1.isna().sum() * 100 / len(temp1)

POSTCODE    0.000000
RPS12       0.000000
RPS13       0.000000
RPS14       0.000000
RPS15       0.013082
RPS16       0.000000
dtype: float64

In [105]:
df1 = df[df['PropertyStyle'] == '03_CONDO']

In [106]:
temp2['POSTCODE'] = df1['FSALDU']
temp2['RPS23'] = df1['LowerBound']
temp2['RPS24'] = df1['ValueEstimate']
temp2['RPS25'] = df1['UpperBound']
temp2['RPS26'] = df1['Avg_BED']
temp2['RPS27'] = df1['Avg_BATH']
# temp2['RPS28'] = df1['Avg_LIVING_AREA']
# temp2['RPS29'] = df1['Avg_LOTSIZE']
# temp2['RPS30'] = df1['Avg_YEARBUILT']
# temp2['RPS31'] = df1['Avg_PARKINGCODE']
# temp2['RPS32'] = df1['Homogeneity']
# temp2['RPS33'] = df1['Resilience_Score']

In [107]:
temp2.isna().sum() * 100 / len(temp2)

POSTCODE    0.000000
RPS23       0.000000
RPS24       0.000000
RPS25       0.000000
RPS26       0.016192
RPS27       0.000000
dtype: float64

In [108]:
df1 = df[df['PropertyStyle'] == '04_ROW']

In [109]:
temp3['POSTCODE'] = df1['FSALDU']
temp3['RPS34'] = df1['LowerBound']
temp3['RPS35'] = df1['ValueEstimate']
temp3['RPS36'] = df1['UpperBound']
temp3['RPS37'] = df1['Avg_BED']
temp3['RPS38'] = df1['Avg_BATH']
# temp3['RPS39'] = df1['Avg_LIVING_AREA']
# temp3['RPS40'] = df1['Avg_LOTSIZE']
# temp3['RPS41'] = df1['Avg_YEARBUILT']
# temp3['RPS42'] = df1['Avg_PARKINGCODE']
# temp3['RPS43'] = df1['Homogeneity']
# temp3['RPS44'] = df1['Resilience_Score']

In [110]:
temp3.isna().sum() * 100 / len(temp3)

POSTCODE    0.000000
RPS34       0.000000
RPS35       0.000000
RPS36       0.000000
RPS37       0.030321
RPS38       0.000000
dtype: float64

In [111]:
len(set(temp3['POSTCODE'].values.tolist()).intersection(set(temp['POSTCODE'].values.tolist())))

92627

In [112]:
print("Postal code with Detached: ", len(set(temp['POSTCODE'].values.tolist())))
print("Postal code with Semi Detached: ",len(set(temp1['POSTCODE'].values.tolist())))
print("Postal code with Condo: ",len(set(temp2['POSTCODE'].values.tolist())))
print("Postal code with Row: ",len(set(temp3['POSTCODE'].values.tolist())))
print("Postal codes with both Condo and Detached: ", len(set(temp2['POSTCODE'].values.tolist()).intersection(set(temp['POSTCODE'].values.tolist()))))

Postal code with Detached:  801238
Postal code with Semi Detached:  76439
Postal code with Condo:  129692
Postal code with Row:  105536
Postal codes with both Condo and Detached:  88818


In [113]:
df.columns

Index(['FSALDU', 'PRCDDA', 'DateYYYYMM', 'CMA_Name', 'PropertyStyle',
       'Min_LowerBound', 'LowerBound', 'ValueEstimate', 'UpperBound',
       'Max_UpperBound', 'Avg_BED', 'StdDev_BED', 'Avg_BATH', 'StdDev_BATH',
       'Avg_LIVING_AREA', 'StdDev_LIVING_AREA', 'Avg_LOTSIZE',
       'StdDev_LOTSIZE', 'Avg_YEARBUILT', 'StdDev_YEARBUILT',
       'Avg_PARKINGCODE', 'Mode_PARKINGTYPE', 'Mode_BASEMENT',
       'Prox_Accommodation_Score', 'Prox_Airports_Score', 'Prox_Cinemas_Score',
       'Prox_Education_Score', 'Prox_Financial_Score', 'Prox_Health_Score',
       'Prox_Water_Score', 'Prox_Golf_Score', 'Prox_ShoppingCentre_Score',
       'Prox_Stadium_Score', 'Prox_Commerical_Score', 'Homogeneity',
       'Resilience_Score'],
      dtype='object')

### Adding the ranking features

In [114]:
proxlist = ['Prox_Accommodation_Score', 'Prox_Airports_Score', 'Prox_Cinemas_Score',
       'Prox_Education_Score', 'Prox_Financial_Score', 'Prox_Health_Score','Prox_Water_Score', 'Prox_Golf_Score', 
        'Prox_ShoppingCentre_Score','Prox_Stadium_Score', 'Prox_Commerical_Score']
#combine: finance shoopping commercial to commercial
#combine: cinemas water golf stadium to entertainment

In [115]:
df[proxlist].head()

,Prox_Accommodation_Score,Prox_Airports_Score,Prox_Cinemas_Score,Prox_Education_Score,Prox_Financial_Score,Prox_Health_Score,Prox_Water_Score,Prox_Golf_Score,Prox_ShoppingCentre_Score,Prox_Stadium_Score,Prox_Commerical_Score
0,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821
1,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821
2,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821
3,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821
4,1.603172,9.56136,0.64223,1.36978,1.054341,2.466794,0.449139,5.798596,1.871097,75.0,0.397821


In [116]:
df[proxlist].isna().sum() * 100 / len(df[proxlist])

Prox_Accommodation_Score     0.0
Prox_Airports_Score          0.0
Prox_Cinemas_Score           0.0
Prox_Education_Score         0.0
Prox_Financial_Score         0.0
Prox_Health_Score            0.0
Prox_Water_Score             0.0
Prox_Golf_Score              0.0
Prox_ShoppingCentre_Score    0.0
Prox_Stadium_Score           0.0
Prox_Commerical_Score        0.0
dtype: float64

In [117]:
for prox in proxlist:
    print(prox)
    print(round(df[prox].max(),3),round(df[prox].median(),3) ,round(df[prox].min(),3), (df[prox].isnull()).sum())

Prox_Accommodation_Score
75.0 3.464 0.044 0
Prox_Airports_Score
75.0 5.112 0.099 0
Prox_Cinemas_Score
75.0 2.753 0.033 0
Prox_Education_Score
75.0 0.689 0.02 0
Prox_Financial_Score
75.0 1.025 0.025 0
Prox_Health_Score
75.0 1.849 0.018 0
Prox_Water_Score
49.495 0.907 0.018 0
Prox_Golf_Score
75.0 2.603 0.004 0
Prox_ShoppingCentre_Score
75.0 0.995 0.003 0
Prox_Stadium_Score
75.0 4.983 0.005 0
Prox_Commerical_Score
75.0 0.29 0.01 0


In [118]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[proxlist] = pd.DataFrame(scaler.fit_transform(df[proxlist]), columns=df[proxlist].columns)

In [119]:
df[proxlist].head()

,Prox_Accommodation_Score,Prox_Airports_Score,Prox_Cinemas_Score,Prox_Education_Score,Prox_Financial_Score,Prox_Health_Score,Prox_Water_Score,Prox_Golf_Score,Prox_ShoppingCentre_Score,Prox_Stadium_Score,Prox_Commerical_Score
0,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176
1,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176
2,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176
3,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176
4,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176


In [120]:
df.head()

,FSALDU,PRCDDA,DateYYYYMM,CMA_Name,PropertyStyle,Min_LowerBound,LowerBound,ValueEstimate,UpperBound,Max_UpperBound,Avg_BED,StdDev_BED,Avg_BATH,StdDev_BATH,Avg_LIVING_AREA,StdDev_LIVING_AREA,Avg_LOTSIZE,StdDev_LOTSIZE,Avg_YEARBUILT,StdDev_YEARBUILT,Avg_PARKINGCODE,Mode_PARKINGTYPE,Mode_BASEMENT,Prox_Accommodation_Score,Prox_Airports_Score,Prox_Cinemas_Score,Prox_Education_Score,Prox_Financial_Score,Prox_Health_Score,Prox_Water_Score,Prox_Golf_Score,Prox_ShoppingCentre_Score,Prox_Stadium_Score,Prox_Commerical_Score,Homogeneity,Resilience_Score
0,A1E2E2,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176,0.547372,0.621751
1,A1E3Z8,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176,0.547372,0.621751
2,A1E4A1,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176,0.547372,0.621751
3,A1E4A5,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176,0.547372,0.621751
4,A1E4A6,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.020801,0.126335,0.008129,0.018,0.01373,0.032664,0.008714,0.07726,0.024911,1.0,0.005176,0.547372,0.621751


In [121]:
#combine: finance shoopping commercial to commercial
#combine: cinemas water golf stadium to 
df['Prox_Commerical_Score'] = (df['Prox_Commerical_Score'] + df['Prox_ShoppingCentre_Score'] + df['Prox_Financial_Score'])/3

In [122]:
proxlist = ['Prox_Education_Score',  'Prox_Health_Score', 'Prox_Commerical_Score', 'Prox_Entertainment_Score']

In [123]:
df['Prox_Entertainment_Score'] = (df['Prox_Cinemas_Score'] + df['Prox_Water_Score'] + df['Prox_Golf_Score'] + df['Prox_Stadium_Score'])/4

In [124]:
df.drop(columns = ['Prox_Cinemas_Score','Prox_Water_Score','Prox_Golf_Score','Prox_Stadium_Score','Prox_ShoppingCentre_Score','Prox_Financial_Score'], inplace = True )

In [125]:
df.drop(columns = ['Prox_Airports_Score','Prox_Accommodation_Score'], inplace = True )

In [126]:
df.head()

,FSALDU,PRCDDA,DateYYYYMM,CMA_Name,PropertyStyle,Min_LowerBound,LowerBound,ValueEstimate,UpperBound,Max_UpperBound,Avg_BED,StdDev_BED,Avg_BATH,StdDev_BATH,Avg_LIVING_AREA,StdDev_LIVING_AREA,Avg_LOTSIZE,StdDev_LOTSIZE,Avg_YEARBUILT,StdDev_YEARBUILT,Avg_PARKINGCODE,Mode_PARKINGTYPE,Mode_BASEMENT,Prox_Education_Score,Prox_Health_Score,Prox_Commerical_Score,Homogeneity,Resilience_Score,Prox_Entertainment_Score
0,A1E2E2,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.018,0.032664,0.014606,0.547372,0.621751,0.273526
1,A1E3Z8,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.018,0.032664,0.014606,0.547372,0.621751,0.273526
2,A1E4A1,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.018,0.032664,0.014606,0.547372,0.621751,0.273526
3,A1E4A5,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.018,0.032664,0.014606,0.547372,0.621751,0.273526
4,A1E4A6,10010165,202010,St. John's,01_SFD,181603.53,181603.53,317984.0,524050.25,524050.25,3.375,0.76967,1.875,0.612372,1438.791667,387.915497,6890.458333,2360.865777,1983.416667,7.546474,1.0,DRIVEWAY-SURFACE,-1,0.018,0.032664,0.014606,0.547372,0.621751,0.273526


In [127]:
def small(x,y):
    if x == y:
        return x
    else:
        return y - x
def percentile100(x): return np.percentile(x, q=100)
def percentile33(x): return np.percentile(x, q=33)
def percentile66(x): return np.percentile(x, q=66)
def rank(Actual, Min,Max, P33, P66):
        if Min == Max:
            return "Same"
        elif Actual < P33:
            return "Low"
        elif Actual < P66:
            return "Medium"
        else:
            return "High"

In [128]:
def Prox_rank(df1, attribute = 'Prox_Accommodation_Score'):
    temp4 = df1.groupby('FSA').agg({attribute: ['min', 'max', percentile33, percentile66]})
    temp4 = temp4.reset_index()
    tempp = pd.DataFrame()
    tempp['FSA'] = temp4['FSA'].values
    tempp['min'] = temp4[attribute]['min'].values
    tempp['max'] = temp4[attribute]['max'].values
    tempp['percentile33'] = temp4[attribute]['percentile33'].values
    tempp['percentile66'] = temp4[attribute]['percentile66'].values
    df1 = pd.merge(df1,tempp, on ='FSA', how = 'left')
    df1[attribute + '_rank'] = df1.apply(lambda x: rank(x[attribute],x['min'],x['max'],x['percentile33'],x['percentile66']),axis = 1)
#     df1.drop(columns = ['min', 'max', 'percentile33', 'percentile66'], inplace = True)
    return df1[attribute + '_rank']

In [129]:
df1 = df[df['PropertyStyle'] == '01_SFD'][['FSALDU'] + proxlist]
df1['FSA'] = df1['FSALDU'].apply(lambda x: x[0:3])

In [130]:
for attribute in proxlist:
    df1[attribute + '_rank'] = Prox_rank(df1[['FSALDU',"FSA",attribute]], attribute = attribute).values
    print(attribute)#recheck the .values logic in above line

Prox_Education_Score
Prox_Health_Score
Prox_Commerical_Score
Prox_Entertainment_Score


In [131]:
ran = df1.iloc[:5,:]

In [132]:
ran['Prox_Education_Score'].values

array([0.01799986, 0.01799986, 0.01799986, 0.01799986, 0.01799986])

In [133]:
np.array([2,3,4,5,6])

array([2, 3, 4, 5, 6])

In [134]:
ran.loc[:,'ranran'] = np.array([2,3,4,5,6])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [135]:
ran

,FSALDU,Prox_Education_Score,Prox_Health_Score,Prox_Commerical_Score,Prox_Entertainment_Score,FSA,Prox_Education_Score_rank,Prox_Health_Score_rank,Prox_Commerical_Score_rank,Prox_Entertainment_Score_rank,ranran
0,A1E2E2,0.018,0.032664,0.014606,0.273526,A1E,High,High,High,Medium,2
1,A1E3Z8,0.018,0.032664,0.014606,0.273526,A1E,High,High,High,Medium,3
2,A1E4A1,0.018,0.032664,0.014606,0.273526,A1E,High,High,High,Medium,4
3,A1E4A5,0.018,0.032664,0.014606,0.273526,A1E,High,High,High,Medium,5
4,A1E4A6,0.018,0.032664,0.014606,0.273526,A1E,High,High,High,Medium,6


In [136]:
# Prox_rank(df1[['FSALDU',"FSA",attribute]], attribute = 'Prox_Education_Score').isna().sum()

In [137]:
# df1[['FSALDU',"FSA",attribute]], attribute = 'Prox_Education_Score')

In [138]:
# df1.tail()

In [139]:
# df1['Prox_Entertainment_Score_rank'].value_counts()

In [140]:
df1.isna().sum()

FSALDU                           0
Prox_Education_Score             0
Prox_Health_Score                0
Prox_Commerical_Score            0
Prox_Entertainment_Score         0
FSA                              0
Prox_Education_Score_rank        0
Prox_Health_Score_rank           0
Prox_Commerical_Score_rank       0
Prox_Entertainment_Score_rank    0
dtype: int64

In [141]:
temp4 = pd.DataFrame()
temp4['POSTCODE'] = df1['FSALDU']
start = 45 + 2
for i in range(0,4):
#     print('RPS' + str(45 + i))
    temp4['RPS' + str(start + i)] = df1[proxlist[i]+'_rank']
#     print(proxlist[i]+'_rank') 

In [142]:
# temp4 = temp4.drop(columns = ['RPS45','RPS46'])

In [143]:
temp4.columns

Index(['POSTCODE', 'RPS47', 'RPS48', 'RPS49', 'RPS50'], dtype='object')

In [144]:
temp4.isna().sum()

POSTCODE    0
RPS47       0
RPS48       0
RPS49       0
RPS50       0
dtype: int64

In [145]:
df1 = df[df['PropertyStyle'] == '02_SD'][['FSALDU'] + proxlist]
df1['FSA'] = df1['FSALDU'].apply(lambda x: x[0:3])

In [146]:
for attribute in proxlist:
    df1[attribute + '_rank'] = Prox_rank(df1[['FSALDU',"FSA",attribute]], attribute = attribute).values
    print(attribute)

Prox_Education_Score
Prox_Health_Score
Prox_Commerical_Score
Prox_Entertainment_Score


In [147]:
df1.head()

,FSALDU,Prox_Education_Score,Prox_Health_Score,Prox_Commerical_Score,Prox_Entertainment_Score,FSA,Prox_Education_Score_rank,Prox_Health_Score_rank,Prox_Commerical_Score_rank,Prox_Entertainment_Score_rank
418,A1A6A8,0.01185,0.011436,0.009981,0.275985,A1A,Low,Low,Low,High
420,A1E1Y7,0.01185,0.011436,0.009981,0.275985,A1E,High,Low,Low,High
422,A1E1Z5,0.01185,0.011436,0.009981,0.275985,A1E,High,Low,Low,High
424,A1E1Z6,0.01185,0.011436,0.009981,0.275985,A1E,High,Low,Low,High
426,A1E2A1,0.01185,0.011436,0.009981,0.275985,A1E,High,Low,Low,High


In [148]:
temp5 = pd.DataFrame()
temp5['POSTCODE'] = df1['FSALDU']
start = 56 +2
for i in range(0,4):
    temp5['RPS' + str(start + i)] = df1[proxlist[i]+'_rank']

In [149]:
temp5.columns

Index(['POSTCODE', 'RPS58', 'RPS59', 'RPS60', 'RPS61'], dtype='object')

In [150]:
temp5.isna().sum()

POSTCODE    0
RPS58       0
RPS59       0
RPS60       0
RPS61       0
dtype: int64

In [151]:
df1 = df[df['PropertyStyle'] == '03_CONDO'][['FSALDU'] + proxlist]
df1['FSA'] = df1['FSALDU'].apply(lambda x: x[0:3])

In [152]:
for attribute in proxlist:
    df1[attribute + '_rank'] = Prox_rank(df1[['FSALDU',"FSA",attribute]], attribute = attribute).values
    print(attribute)

Prox_Education_Score
Prox_Health_Score
Prox_Commerical_Score
Prox_Entertainment_Score


In [153]:
df1.tail()

,FSALDU,Prox_Education_Score,Prox_Health_Score,Prox_Commerical_Score,Prox_Entertainment_Score,FSA,Prox_Education_Score_rank,Prox_Health_Score_rank,Prox_Commerical_Score_rank,Prox_Entertainment_Score_rank
1112708,X1A1Y6,0.003757,0.012398,0.33792,0.511351,X1A,Same,Same,Same,Same
1112710,X1A2B3,0.003757,0.012398,0.33792,0.511351,X1A,Same,Same,Same,Same
1112712,X1A2R7,0.003757,0.012398,0.33792,0.511351,X1A,Same,Same,Same,Same
1112714,X1A2V2,0.003757,0.012398,0.33792,0.511351,X1A,Same,Same,Same,Same
1112716,X1A3Z5,0.003757,0.012398,0.33792,0.511351,X1A,Same,Same,Same,Same


In [154]:
temp6 = pd.DataFrame()
temp6['POSTCODE'] = df1['FSALDU']
start = 67 + 2
for i in range(0,4):
    temp6['RPS' + str(start + i)] = df1[proxlist[i]+'_rank']

In [155]:
temp6.columns

Index(['POSTCODE', 'RPS69', 'RPS70', 'RPS71', 'RPS72'], dtype='object')

In [156]:
df1 = df[df['PropertyStyle'] == '04_ROW'][['FSALDU'] + proxlist]
df1['FSA'] = df1['FSALDU'].apply(lambda x: x[0:3])

In [157]:
for attribute in proxlist:
    df1[attribute + '_rank'] = Prox_rank(df1[['FSALDU',"FSA",attribute]], attribute = attribute).values
    print(attribute)

Prox_Education_Score
Prox_Health_Score
Prox_Commerical_Score
Prox_Entertainment_Score


In [158]:
df1.tail()

,FSALDU,Prox_Education_Score,Prox_Health_Score,Prox_Commerical_Score,Prox_Entertainment_Score,FSA,Prox_Education_Score_rank,Prox_Health_Score_rank,Prox_Commerical_Score_rank,Prox_Entertainment_Score_rank
1112446,Y1A5S4,0.022206,0.087987,0.364634,0.523995,Y1A,Medium,Medium,High,Medium
1112448,Y1A5S5,0.022206,0.087987,0.364634,0.523995,Y1A,Medium,Medium,High,Medium
1112450,Y1A5W4,0.022206,0.087987,0.364634,0.523995,Y1A,Medium,Medium,High,Medium
1112452,Y1A5W5,0.022206,0.087987,0.364634,0.523995,Y1A,Medium,Medium,High,Medium
1112454,Y1A6N2,0.022206,0.087987,0.364634,0.523995,Y1A,Medium,Medium,High,Medium


In [159]:
temp7 = pd.DataFrame()
temp7['POSTCODE'] = df1['FSALDU']
start = 78 + 2
for i in range(0,4):
    temp7['RPS' + str(start + i)] = df1[proxlist[i]+'_rank']

In [160]:
temp7.columns

Index(['POSTCODE', 'RPS80', 'RPS81', 'RPS82', 'RPS83'], dtype='object')

##### merging RPS dataframes

In [161]:
temp.head()

,POSTCODE,RPS1,RPS2,RPS3,RPS4,RPS5
0,A1E2E2,181603.53,317984.0,524050.25,3.375,1.875
1,A1E3Z8,181603.53,317984.0,524050.25,3.375,1.875
2,A1E4A1,181603.53,317984.0,524050.25,3.375,1.875
3,A1E4A5,181603.53,317984.0,524050.25,3.375,1.875
4,A1E4A6,181603.53,317984.0,524050.25,3.375,1.875


In [162]:
df.FSALDU.nunique(),temp.POSTCODE.nunique(),temp1.POSTCODE.nunique(),temp4.POSTCODE.nunique(),temp5.POSTCODE.nunique(),

(847683, 801238, 76439, 801238, 76439)

In [163]:
temp1.head()

,POSTCODE,RPS12,RPS13,RPS14,RPS15,RPS16
418,A1A6A8,217844.374,262482.53,285610.914,3.181818,1.272727
420,A1E1Y7,217844.374,262482.53,285610.914,3.181818,1.272727
422,A1E1Z5,217844.374,262482.53,285610.914,3.181818,1.272727
424,A1E1Z6,217844.374,262482.53,285610.914,3.181818,1.272727
426,A1E2A1,217844.374,262482.53,285610.914,3.181818,1.272727


In [164]:
temp.isna().sum()

POSTCODE    0
RPS1        0
RPS2        0
RPS3        0
RPS4        0
RPS5        0
dtype: int64

In [165]:
temp = pd.merge(temp,temp1, on ='POSTCODE', how = 'outer')

In [166]:
temp.isna().sum()

POSTCODE         0
RPS1          2064
RPS2          2064
RPS3          2064
RPS4          2064
RPS5          2064
RPS12       726863
RPS13       726863
RPS14       726863
RPS15       726873
RPS16       726863
dtype: int64

In [167]:
temp.head()

,POSTCODE,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16
0,A1E2E2,181603.53,317984.0,524050.25,3.375,1.875,NaN,NaN,NaN,NaN,NaN
1,A1E3Z8,181603.53,317984.0,524050.25,3.375,1.875,NaN,NaN,NaN,NaN,NaN
2,A1E4A1,181603.53,317984.0,524050.25,3.375,1.875,NaN,NaN,NaN,NaN,NaN
3,A1E4A5,181603.53,317984.0,524050.25,3.375,1.875,NaN,NaN,NaN,NaN,NaN
4,A1E4A6,181603.53,317984.0,524050.25,3.375,1.875,NaN,NaN,NaN,NaN,NaN


In [168]:
temp2 = temp2.reset_index(drop = True)

In [169]:
temp = pd.merge(temp,temp2, on ='POSTCODE', how = 'outer')

In [170]:
temp = pd.merge(temp,temp3, on ='POSTCODE', how = 'outer')

In [171]:
temp = pd.merge(temp,temp4, on ='POSTCODE', how = 'outer')

In [172]:
temp = pd.merge(temp,temp5, on ='POSTCODE', how = 'outer')

In [173]:
temp = pd.merge(temp,temp6, on ='POSTCODE', how = 'outer')

In [174]:
temp = pd.merge(temp,temp7, on ='POSTCODE', how = 'outer')

In [175]:
temp.shape

(847683, 37)

In [176]:
temp = temp.round(2)

In [177]:
temp['POSTCODE'].nunique(), temp.shape

(847683, (847683, 37))

In [178]:
temp['FSA'] = temp['POSTCODE'].apply(lambda x: x[:3])

In [179]:
temp.head()

,POSTCODE,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38,RPS47,RPS48,RPS49,RPS50,RPS58,RPS59,RPS60,RPS61,RPS69,RPS70,RPS71,RPS72,RPS80,RPS81,RPS82,RPS83,FSA
0,A1E2E2,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
1,A1E3Z8,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
2,A1E4A1,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
3,A1E4A5,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
4,A1E4A6,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,Medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E


In [180]:
temp.columns[21:-1]

Index(['RPS47', 'RPS48', 'RPS49', 'RPS50', 'RPS58', 'RPS59', 'RPS60', 'RPS61',
       'RPS69', 'RPS70', 'RPS71', 'RPS72', 'RPS80', 'RPS81', 'RPS82', 'RPS83'],
      dtype='object')

In [181]:
temprank = temp[temp.columns[21:-1]]

In [182]:
temprank = temprank.fillna("Medium")

In [183]:
# temprank.isna().sum()

In [184]:
temp = temp.drop(columns =temp.columns[21:-1] )

In [185]:
temp.columns[1:-1]

Index(['RPS1', 'RPS2', 'RPS3', 'RPS4', 'RPS5', 'RPS12', 'RPS13', 'RPS14',
       'RPS15', 'RPS16', 'RPS23', 'RPS24', 'RPS25', 'RPS26', 'RPS27', 'RPS34',
       'RPS35', 'RPS36', 'RPS37', 'RPS38'],
      dtype='object')

In [186]:
aggdict = {}
for col in temp.columns[1:-1]:
    aggdict[col] = 'mean'

In [187]:
fsa = temp.groupby('FSA').agg(aggdict).reset_index()

In [188]:
fsa.head()

,FSA,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38
0,A0A,117317.593908,216284.283103,442193.309885,2.936207,1.383563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0B,109072.837794,188679.131618,304119.040735,2.869412,1.326912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0C,88174.476970,178015.276364,335039.956970,3.046061,1.402727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0E,112448.763704,208202.084074,419118.263704,3.032778,1.279815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0G,83507.646744,168969.430698,377756.250349,2.925814,1.287209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
fsa.isna().sum()

FSA        0
RPS1      17
RPS2      17
RPS3      17
RPS4      17
RPS5      17
RPS12    802
RPS13    802
RPS14    802
RPS15    802
RPS16    802
RPS23    516
RPS24    516
RPS25    516
RPS26    516
RPS27    516
RPS34    656
RPS35    656
RPS36    656
RPS37    657
RPS38    656
dtype: int64

In [190]:
fsa['F'] = fsa['FSA'].apply(lambda x: x[0])

In [191]:
aggdict = {}
for col in fsa.columns[1:-1]:
    aggdict[col] = 'mean'

In [192]:
fsa.columns[1:-1]

Index(['RPS1', 'RPS2', 'RPS3', 'RPS4', 'RPS5', 'RPS12', 'RPS13', 'RPS14',
       'RPS15', 'RPS16', 'RPS23', 'RPS24', 'RPS25', 'RPS26', 'RPS27', 'RPS34',
       'RPS35', 'RPS36', 'RPS37', 'RPS38'],
      dtype='object')

In [193]:
fsa['F'].unique()

array(['A', 'B', 'C', 'E', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'R',
       'S', 'T', 'V', 'X', 'Y'], dtype=object)

In [194]:
f = fsa.groupby('F').agg(aggdict).reset_index()

In [195]:
f

,F,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38
0,A,1.696368e+05,2.909158e+05,4.787461e+05,3.007562,1.549231,1.994813e+05,2.486665e+05,3.066032e+05,2.989986,1.441572,243831.779444,329348.763204,469064.943315,2.317658,1.750215,193083.475525,244503.312965,3.032218e+05,2.809014,1.431514
1,B,1.811044e+05,2.988405e+05,4.712065e+05,2.919644,1.476788,2.280502e+05,2.797425e+05,3.288308e+05,2.742079,1.479846,229664.622883,325393.457994,429324.146328,1.951896,1.590857,279724.709968,354847.947007,4.345126e+05,3.142483,1.775644
2,C,1.931751e+05,3.250923e+05,4.930065e+05,3.109217,1.517812,1.973528e+05,2.978301e+05,3.597701e+05,2.504651,1.314419,226885.706087,297485.522957,463139.131478,2.016696,1.171565,340577.857765,442616.110625,5.524113e+05,4.029667,2.231776
3,E,1.273192e+05,2.367670e+05,4.104885e+05,2.884406,1.385346,1.831454e+05,2.444649e+05,2.805611e+05,2.638527,1.408733,178626.748043,263703.370850,369109.029586,2.057849,1.489360,187807.776371,234628.350481,3.320737e+05,3.455722,1.780967
4,G,1.652511e+05,2.743304e+05,4.232134e+05,2.775342,1.290484,1.839014e+05,2.323894e+05,2.813063e+05,2.222199,1.142294,181590.713102,263086.767551,349528.734999,2.052569,1.140183,194696.448856,248062.250037,3.141129e+05,2.312258,1.154964
5,H,5.486513e+05,8.010900e+05,1.054403e+06,3.642523,1.872739,5.694164e+05,7.339287e+05,8.555409e+05,3.369592,1.558094,292053.625821,436405.996824,626164.016579,1.999039,1.244219,522088.158785,684537.246966,8.486349e+05,2.755555,1.419949
6,J,2.427548e+05,3.796537e+05,5.687703e+05,2.778167,1.366894,2.621437e+05,3.135133e+05,3.681401e+05,2.552625,1.147578,206984.551396,295653.729635,376122.392571,2.124272,1.143394,259059.167969,329088.789067,4.168150e+05,2.430879,1.199476
7,K,3.963195e+05,6.050511e+05,8.495744e+05,3.170950,1.831274,4.201144e+05,4.974930e+05,5.953859e+05,2.953271,1.589394,271520.552823,364484.636138,478492.420804,1.990544,1.418753,365864.161129,467191.388043,5.621656e+05,2.889167,1.731907
8,L,6.701004e+05,9.553630e+05,1.295228e+06,3.328184,2.076945,6.138887e+05,6.882978e+05,7.362496e+05,3.099157,1.809680,372525.532996,482833.061964,585514.214921,1.785226,1.515007,547281.888679,667765.551157,7.596149e+05,2.903531,1.836771
9,M,1.096615e+06,1.530635e+06,1.962319e+06,3.340261,2.176684,1.006461e+06,1.183263e+06,1.327755e+06,3.154912,1.913496,439071.356933,630507.758481,929252.519995,1.718973,1.446782,754315.136621,955301.518529,1.133934e+06,2.791277,1.862401


In [196]:
f.columns[1:]

Index(['RPS1', 'RPS2', 'RPS3', 'RPS4', 'RPS5', 'RPS12', 'RPS13', 'RPS14',
       'RPS15', 'RPS16', 'RPS23', 'RPS24', 'RPS25', 'RPS26', 'RPS27', 'RPS34',
       'RPS35', 'RPS36', 'RPS37', 'RPS38'],
      dtype='object')

In [197]:
for col in f.columns[1:]:
    if len(f.loc[f[col].isna(),col]) > 0:
        f.loc[f[col].isna(),col] = f.loc[f.F == 'Y',col].item()

In [198]:
fsa.loc[(fsa['RPS12'].isna()) & (fsa['F'] == 'C'),'RPS12']

112   NaN
116   NaN
117   NaN
Name: RPS12, dtype: float64

In [199]:
f.loc[f['F'] == 'X','RPS12'].item()
#     fsa.loc[(fsa['RPS12'].isna()) & (fsa['F'] == 'B'),'RPS12'] = f.loc[f['F'] == 'B','RPS12'].item()

310702.3099999997

In [200]:
for col in fsa.columns[1:-1]:
    for colfilter in fsa['F'].unique():
        if len(fsa.loc[(fsa[col].isna()) & (fsa['F'] == colfilter),col]) > 0:
            fsa.loc[(fsa[col].isna()) & (fsa['F'] == colfilter),col] = f.loc[f['F'] == colfilter,col].item()

In [201]:
fsa.isna().sum()

FSA      0
RPS1     0
RPS2     0
RPS3     0
RPS4     0
RPS5     0
RPS12    0
RPS13    0
RPS14    0
RPS15    0
RPS16    0
RPS23    0
RPS24    0
RPS25    0
RPS26    0
RPS27    0
RPS34    0
RPS35    0
RPS36    0
RPS37    0
RPS38    0
F        0
dtype: int64

In [202]:
fsa.shape, temp['FSA'].nunique()

((1637, 22), 1637)

In [203]:
temp

,POSTCODE,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38,FSA
0,A1E2E2,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
1,A1E3Z8,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
2,A1E4A1,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
3,A1E4A5,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
4,A1E4A6,181603.53,317984.0,524050.25,3.38,1.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847678,V1X6E2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380609.23,454677.88,559503.9,3.27,2.27,V1X
847679,V1X6Z3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380609.23,454677.88,559503.9,3.27,2.27,V1X
847680,V1X7H3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380609.23,454677.88,559503.9,3.27,2.27,V1X
847681,V1X8B2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380609.23,454677.88,559503.9,3.27,2.27,V1X


In [204]:
for col in temp.columns[1:-1]:
    for colfilter in temp['FSA'].unique():
        if len(temp.loc[(temp[col].isna()) & (temp['FSA'] == colfilter),col]) > 0:
            temp.loc[(temp[col].isna()) & (temp['FSA'] == colfilter),col] = fsa.loc[fsa['FSA'] == colfilter,col].item()

In [205]:
temp.isna().sum()

POSTCODE    0
RPS1        0
RPS2        0
RPS3        0
RPS4        0
RPS5        0
RPS12       0
RPS13       0
RPS14       0
RPS15       0
RPS16       0
RPS23       0
RPS24       0
RPS25       0
RPS26       0
RPS27       0
RPS34       0
RPS35       0
RPS36       0
RPS37       0
RPS38       0
FSA         0
dtype: int64

In [206]:
tempfinal = pd.concat([temp,temprank],axis = 1)

In [207]:
tempfinal.shape, temp.shape, temprank.shape

((847683, 38), (847683, 22), (847683, 16))

In [209]:
flatfile.shape

(799313, 60)

In [210]:
flatfilejoin = pd.merge(flatfile,tempfinal, on ='POSTCODE', how = 'inner')

In [211]:
flatfilejoin.shape

(764863, 97)

In [212]:
flatfilejoin.head(5)

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,HH1,HH2,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38,FSA,RPS47,RPS48,RPS49,RPS50,RPS58,RPS59,RPS60,RPS61,RPS69,RPS70,RPS71,RPS72,RPS80,RPS81,RPS82,RPS83
0,A0A0B7,12.86,10.59,12.96,21.38,42.21,56.20,52.92,22.43,24.65,34.17,23.66,12.04,25.67,4.52,0.00,0.0,17.67,38.80,24.50,19.24,53852.94,66646.81,76.44,0.00,14.04,0.0,9.77,77.44,22.31,0.00,0.0,100.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,13.40,0.00,20.21,0.00,16.38,22.77,12.34,15.11,100.00,0.00,0.00,0.00,0.0,100.00,0.00,0.0,50.30,49.70,72634.33,114355.52,224611.80,2.86,1.14,199481.275504,248666.525882,306603.227515,2.989986,1.441572,243831.779444,329348.763204,469064.943315,2.317658,1.750215,193083.475525,244503.312965,303221.783803,2.809014,1.431514,A0A,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
1,A0A0C2,10.65,12.04,13.08,25.47,38.81,54.79,54.12,25.45,20.43,30.11,32.84,13.50,15.39,4.81,3.28,0.0,14.87,35.08,26.18,24.19,62993.06,88664.17,95.53,0.00,0.00,0.0,4.47,71.85,6.50,21.52,0.0,100.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,4.44,13.21,0.00,8.88,18.56,38.38,16.40,100.00,0.00,0.00,0.00,0.0,95.08,4.92,0.0,48.31,51.69,72634.33,102677.06,182972.77,2.91,1.18,199481.275504,248666.525882,306603.227515,2.989986,1.441572,243831.779444,329348.763204,469064.943315,2.317658,1.750215,193083.475525,244503.312965,303221.783803,2.809014,1.431514,A0A,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
2,A0A1B0,13.63,16.30,16.58,21.12,32.36,48.48,48.77,27.52,23.71,12.13,15.10,28.46,30.33,10.71,3.27,0.0,5.74,35.49,16.58,42.19,83602.78,109506.70,88.87,3.57,3.94,0.0,3.62,18.33,57.58,24.10,0.0,100.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,5.45,19.77,7.16,3.64,12.68,2.73,14.70,27.53,3.64,2.70,94.81,0.00,3.43,1.75,0.0,79.66,20.34,0.0,48.56,51.44,129402.20,199355.78,555162.90,2.93,1.31,199481.275504,248666.525882,306603.227515,2.989986,1.441572,243831.779444,329348.763204,469064.943315,2.317658,1.750215,193083.475525,244503.312965,303221.783803,2.809014,1.431514,A0A,Low,Medium,Medium,Low,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
3,A0A1C0,17.41,15.16,22.03,19.52,25.89,42.37,55.73,24.00,20.27,21.90,24.62,11.69,25.01,11.97,4.81,0.0,10.63,24.88,21.13,43.36,89877.00,106513.00,95.93,0.00,1.18,0.0,2.89,44.12,46.86,9.02,2.2,97.8,68.51,31.49,0.0,0.0,0.0,0.0,0.0,2.49,19.77,11.07,12.50,9.96,1.79,12.12,17.87,6.85,5.58,96.41,0.63,2.89,0.07,0.0,90.55,9.45,0.0,38.73,61.27,220200.53,440161.20,591706.25,3.00,1.90,199481.275504,248666.525882,306603.227515,2.989986,1.441572,243831.779444,329348.763204,469064.943315,2.317658,1.750215,193083.475525,244503.312965,303221.783803,2.809014,1.431514,A0A,High,Low,Low,Low,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
4,A0A1E0,10.53,11.44,12.24,21.63,44.17,56.60,54.42,24.29,21.30,38.16,34.21,4.95,21.03,1.65,0.00,0.0,11.47,38.25,30.50,19.78,60681.00,67548.34,92.31,0.00,0.00,0.0,7.69,66.67,7.41,25.93,0.0,100.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,5.83,14.44,5.75,0.00,0.00,0.00,9.45,12.28,24.01,28.24,100.00,0.00,0.00,0.00,0.0,91.89,8.11,0.0,52.00,48.00,72634.33,102787.08,182972.77,3.00,1.50,199481.275504,248666.525882,306603.227515,2.989986,1.441572,243831.779444,329348.763204,469064.943315,2.317658,1.750215,193083.475525,244503.312965,303221.783803,2.809014,1.431514,A0A,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium


In [213]:
flatfilejoin.isna().sum()

POSTCODE    0
PP1         0
PP2         0
PP3         0
PP4         0
PP5         0
PP6         0
PP7         0
PP8         0
PP9         0
ED1         0
ED2         0
ED3         0
ED4         0
ED5         0
ED6         0
ED7         0
IN1         0
IN2         0
IN3         0
IN4         0
IN5         0
IN6         0
EM1         0
EM2         0
EM3         0
EM4         0
EM5         0
EM6         0
EM7         0
EM8         0
MN1         0
MN2         0
MN3         0
MN4         0
MN5         0
MN6         0
MN7         0
MN8         0
MN9         0
OC1         0
OC2         0
OC3         0
OC4         0
OC5         0
OC6         0
OC7         0
OC8         0
OC9         0
OC10        0
DW1         0
DW2         0
DW3         0
DW4         0
DW5         0
DW6         0
DW7         0
DW8         0
HH1         0
HH2         0
RPS1        0
RPS2        0
RPS3        0
RPS4        0
RPS5        0
RPS12       0
RPS13       0
RPS14       0
RPS15       0
RPS16       0
RPS23       0
RPS24 

In [214]:
len(set(flatfilejoin.POSTCODE.unique()).intersection(set(temp.POSTCODE.unique())))

764863

In [215]:
len(set(flatfilejoin.POSTCODE.unique()))

764863

In [231]:
flatfilejoin = flatfilejoin.drop(columns = ['FSA'])

In [240]:
flatfilejoin.dtypes.keys()

Index(['POSTCODE', 'PP1', 'PP2', 'PP3', 'PP4', 'PP5', 'PP6', 'PP7', 'PP8',
       'PP9', 'ED1', 'ED2', 'ED3', 'ED4', 'ED5', 'ED6', 'ED7', 'IN1', 'IN2',
       'IN3', 'IN4', 'IN5', 'IN6', 'EM1', 'EM2', 'EM3', 'EM4', 'EM5', 'EM6',
       'EM7', 'EM8', 'MN1', 'MN2', 'MN3', 'MN4', 'MN5', 'MN6', 'MN7', 'MN8',
       'MN9', 'OC1', 'OC2', 'OC3', 'OC4', 'OC5', 'OC6', 'OC7', 'OC8', 'OC9',
       'OC10', 'DW1', 'DW2', 'DW3', 'DW4', 'DW5', 'DW6', 'DW7', 'DW8', 'HH1',
       'HH2', 'RPS1', 'RPS2', 'RPS3', 'RPS4', 'RPS5', 'RPS12', 'RPS13',
       'RPS14', 'RPS15', 'RPS16', 'RPS23', 'RPS24', 'RPS25', 'RPS26', 'RPS27',
       'RPS34', 'RPS35', 'RPS36', 'RPS37', 'RPS38', 'RPS47', 'RPS48', 'RPS49',
       'RPS50', 'RPS58', 'RPS59', 'RPS60', 'RPS61', 'RPS69', 'RPS70', 'RPS71',
       'RPS72', 'RPS80', 'RPS81', 'RPS82', 'RPS83'],
      dtype='object')

In [250]:
flatfilejoin.dtypes['POSTCODE']

dtype('O')

In [249]:
flatfilejoin.dtypes['POSTCODE'] == 'float'

False

In [252]:
convert_dict = {}
for key in flatfilejoin.dtypes.keys():
    if flatfilejoin.dtypes[key] == float:
        convert_dict[key] = int

In [253]:
convert_dict

{'PP1': int,
 'PP2': int,
 'PP3': int,
 'PP4': int,
 'PP5': int,
 'PP6': int,
 'PP7': int,
 'PP8': int,
 'PP9': int,
 'ED1': int,
 'ED2': int,
 'ED3': int,
 'ED4': int,
 'ED5': int,
 'ED6': int,
 'ED7': int,
 'IN1': int,
 'IN2': int,
 'IN3': int,
 'IN4': int,
 'IN5': int,
 'IN6': int,
 'EM1': int,
 'EM2': int,
 'EM3': int,
 'EM4': int,
 'EM5': int,
 'EM6': int,
 'EM7': int,
 'EM8': int,
 'MN1': int,
 'MN2': int,
 'MN3': int,
 'MN4': int,
 'MN5': int,
 'MN6': int,
 'MN7': int,
 'MN8': int,
 'MN9': int,
 'OC1': int,
 'OC2': int,
 'OC3': int,
 'OC4': int,
 'OC5': int,
 'OC6': int,
 'OC7': int,
 'OC8': int,
 'OC9': int,
 'OC10': int,
 'DW1': int,
 'DW2': int,
 'DW3': int,
 'DW4': int,
 'DW5': int,
 'DW6': int,
 'DW7': int,
 'DW8': int,
 'HH1': int,
 'HH2': int,
 'RPS1': int,
 'RPS2': int,
 'RPS3': int,
 'RPS4': int,
 'RPS5': int,
 'RPS12': int,
 'RPS13': int,
 'RPS14': int,
 'RPS15': int,
 'RPS16': int,
 'RPS23': int,
 'RPS24': int,
 'RPS25': int,
 'RPS26': int,
 'RPS27': int,
 'RPS34': in

In [254]:
flatfilejoin = flatfilejoin.astype(convert_dict) 

In [255]:
flatfilejoin

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,HH1,HH2,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38,RPS47,RPS48,RPS49,RPS50,RPS58,RPS59,RPS60,RPS61,RPS69,RPS70,RPS71,RPS72,RPS80,RPS81,RPS82,RPS83
0,A0A0B7,12,10,12,21,42,56,52,22,24,34,23,12,25,4,0,0,17,38,24,19,53852,66646,76,0,14,0,9,77,22,0,0,100,0,0,0,0,0,0,0,0,0,13,0,20,0,16,22,12,15,100,0,0,0,0,100,0,0,50,49,72634,114355,224611,2,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
1,A0A0C2,10,12,13,25,38,54,54,25,20,30,32,13,15,4,3,0,14,35,26,24,62993,88664,95,0,0,0,4,71,6,21,0,100,0,0,0,0,0,0,0,0,0,0,4,13,0,8,18,38,16,100,0,0,0,0,95,4,0,48,51,72634,102677,182972,2,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
2,A0A1B0,13,16,16,21,32,48,48,27,23,12,15,28,30,10,3,0,5,35,16,42,83602,109506,88,3,3,0,3,18,57,24,0,100,0,0,0,0,0,0,0,5,19,7,3,12,2,14,27,3,2,94,0,3,1,0,79,20,0,48,51,129402,199355,555162,2,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,Low,Medium,Medium,Low,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
3,A0A1C0,17,15,22,19,25,42,55,24,20,21,24,11,25,11,4,0,10,24,21,43,89877,106513,95,0,1,0,2,44,46,9,2,97,68,31,0,0,0,0,0,2,19,11,12,9,1,12,17,6,5,96,0,2,0,0,90,9,0,38,61,220200,440161,591706,3,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,Low,Low,Low,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
4,A0A1E0,10,11,12,21,44,56,54,24,21,38,34,4,21,1,0,0,11,38,30,19,60681,67548,92,0,0,0,7,66,7,25,0,100,0,0,0,0,0,0,0,5,14,5,0,0,0,9,12,24,28,100,0,0,0,0,91,8,0,52,48,72634,102787,182972,3,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764858,Y1A6W1,22,21,22,23,9,35,56,24,19,12,24,8,26,19,6,2,0,11,19,67,136407,162232,93,2,1,1,1,94,4,1,11,88,28,0,4,3,0,63,0,15,15,9,10,21,3,13,10,1,0,74,18,0,6,0,83,16,0,28,71,439474,636696,763686,2,1,310702,380693,425906,2,1,261968,425965,478626,1,1,330809,496571,586898,2,1,High,High,High,High,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
764859,Y1A7A1,17,15,24,22,20,39,35,31,33,10,17,8,22,25,13,2,3,19,26,50,106899,112263,78,11,7,2,0,94,5,0,10,89,17,0,0,0,0,82,0,5,14,8,10,26,3,17,12,0,1,45,42,9,2,0,70,29,0,35,64,407615,467701,680122,2,1,310702,400491,439794,2,1,240035,375415,418448,1,1,330809,496571,586898,2,1,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,High,High,High,Medium,Medium,Medium,Medium
764860,Y1A7A2,17,15,24,22,20,39,35,31,33,13,16,9,23,23,10,2,5,25,28,40,92550,99121,76,5,15,2,0,93,6,0,6,93,18,0,0,0,0,81,0,5,14,8,10,26,3,17,12,0,1,57,34,5,1,1,66,33,0,40,59,407615,467701,680122,2,1,310702,400491,439794,2,1,240035,375415,418448,1,1,330809,496571,586898,2,1,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,High,High,High,Medium,Medium,Medium,Medium
764861,Y1A7A3,17,15,24,22,20,39,35,31,33,10,17,8,22,25,13,2,3,19,26,50,106784,113973,78,11,7,2,0,94,5,0,10,89,17,0,0,0,0,82,0,5,14,8,10,26,3,17,12,0,1,45,42,9,2,0,70,29,0,35,

# Renaming the keys:

In [20]:
# !pip install openpyxl

In [49]:
df = pd.read_excel('Data Dictionary SuperDemographics 2020.xlsx', sheet_name = 'Final')

In [50]:
df = df.iloc[:,:2]

In [51]:
df = df.rename(columns={"POPULATION": "key", "Unnamed: 1": "value"})

In [52]:
df = df[df.value.notnull()]

In [53]:
df['value'] = df['value'].apply(lambda x: x.strip()).values

In [56]:
df

,key,value
0,PP1,Percentage of total population in this postalc...
1,PP2,Percentage of total population in this postalc...
2,PP3,Percentage of total population in this postalc...
3,PP4,Percentage of total population in this postalc...
4,PP5,Percentage of total population in this postalc...
5,PP6,Median age of total population: half of the po...
6,PP7,Percentage of population above age 15 years ma...
7,PP8,Percentage of population above age 15 years ne...
8,PP9,Percentage of population above age 15 years ot...
10,ED1,% percentage of population in private househol...


In [59]:
rename = {}
for i in range(0,len(df)):
    rename[df.iloc[i,0]] = df.iloc[i,1]

In [60]:
rename

{'PP1': 'Percentage of total population in this postalcode that belongs to people between ages 0 -14 years',
 'PP2': 'Percentage of total population in this postalcode that belongs to people between ages 15 -29 years',
 'PP3': 'Percentage of total population in this postalcode that belongs to people between ages 30 -44 years',
 'PP4': 'Percentage of total population in this postalcode that belongs to people between ages 45 - 59 years',
 'PP5': 'Percentage of total population in this postalcode that belongs to people aged 60 and above',
 'PP6': 'Median age of total population: half of the population below this age, the other half above this age.',
 'PP7': 'Percentage of population above age 15 years married',
 'PP8': 'Percentage of population above age 15 years never married',
 'PP9': 'Percentage of population above age 15 years others (living common law, seperated, divorced and widowed)',
 'ED1': '% percentage of population in private households with No certificate, diploma or degree',

In [67]:
flatfilejoin = pd.read_csv('s3://{}/{}'.format('myhomely/raja/Manifold-selected', 'flatfile' + '.csv')).drop(columns = ['Unnamed: 0'])

In [68]:
flatfilejoin

,POSTCODE,PP1,PP2,PP3,PP4,PP5,PP6,PP7,PP8,PP9,ED1,ED2,ED3,ED4,ED5,ED6,ED7,IN1,IN2,IN3,IN4,IN5,IN6,EM1,EM2,EM3,EM4,EM5,EM6,EM7,EM8,MN1,MN2,MN3,MN4,MN5,MN6,MN7,MN8,MN9,OC1,OC2,OC3,OC4,OC5,OC6,OC7,OC8,OC9,OC10,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,HH1,HH2,RPS1,RPS2,RPS3,RPS4,RPS5,RPS12,RPS13,RPS14,RPS15,RPS16,RPS23,RPS24,RPS25,RPS26,RPS27,RPS34,RPS35,RPS36,RPS37,RPS38,RPS47,RPS48,RPS49,RPS50,RPS58,RPS59,RPS60,RPS61,RPS69,RPS70,RPS71,RPS72,RPS80,RPS81,RPS82,RPS83
0,A0A0B7,12,10,12,21,42,56,52,22,24,34,23,12,25,4,0,0,17,38,24,19,53852,66646,76,0,14,0,9,77,22,0,0,100,0,0,0,0,0,0,0,0,0,13,0,20,0,16,22,12,15,100,0,0,0,0,100,0,0,50,49,72634,114355,224611,2,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
1,A0A0C2,10,12,13,25,38,54,54,25,20,30,32,13,15,4,3,0,14,35,26,24,62993,88664,95,0,0,0,4,71,6,21,0,100,0,0,0,0,0,0,0,0,0,0,4,13,0,8,18,38,16,100,0,0,0,0,95,4,0,48,51,72634,102677,182972,2,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
2,A0A1B0,13,16,16,21,32,48,48,27,23,12,15,28,30,10,3,0,5,35,16,42,83602,109506,88,3,3,0,3,18,57,24,0,100,0,0,0,0,0,0,0,5,19,7,3,12,2,14,27,3,2,94,0,3,1,0,79,20,0,48,51,129402,199355,555162,2,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,Low,Medium,Medium,Low,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
3,A0A1C0,17,15,22,19,25,42,55,24,20,21,24,11,25,11,4,0,10,24,21,43,89877,106513,95,0,1,0,2,44,46,9,2,97,68,31,0,0,0,0,0,2,19,11,12,9,1,12,17,6,5,96,0,2,0,0,90,9,0,38,61,220200,440161,591706,3,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,Low,Low,Low,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
4,A0A1E0,10,11,12,21,44,56,54,24,21,38,34,4,21,1,0,0,11,38,30,19,60681,67548,92,0,0,0,7,66,7,25,0,100,0,0,0,0,0,0,0,5,14,5,0,0,0,9,12,24,28,100,0,0,0,0,91,8,0,52,48,72634,102787,182972,3,1,199481,248666,306603,2,1,243831,329348,469064,2,1,193083,244503,303221,2,1,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764858,Y1A6W1,22,21,22,23,9,35,56,24,19,12,24,8,26,19,6,2,0,11,19,67,136407,162232,93,2,1,1,1,94,4,1,11,88,28,0,4,3,0,63,0,15,15,9,10,21,3,13,10,1,0,74,18,0,6,0,83,16,0,28,71,439474,636696,763686,2,1,310702,380693,425906,2,1,261968,425965,478626,1,1,330809,496571,586898,2,1,High,High,High,High,High,High,High,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,Medium
764859,Y1A7A1,17,15,24,22,20,39,35,31,33,10,17,8,22,25,13,2,3,19,26,50,106899,112263,78,11,7,2,0,94,5,0,10,89,17,0,0,0,0,82,0,5,14,8,10,26,3,17,12,0,1,45,42,9,2,0,70,29,0,35,64,407615,467701,680122,2,1,310702,400491,439794,2,1,240035,375415,418448,1,1,330809,496571,586898,2,1,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,High,High,High,Medium,Medium,Medium,Medium
764860,Y1A7A2,17,15,24,22,20,39,35,31,33,13,16,9,23,23,10,2,5,25,28,40,92550,99121,76,5,15,2,0,93,6,0,6,93,18,0,0,0,0,81,0,5,14,8,10,26,3,17,12,0,1,57,34,5,1,1,66,33,0,40,59,407615,467701,680122,2,1,310702,400491,439794,2,1,240035,375415,418448,1,1,330809,496571,586898,2,1,High,Medium,Medium,Medium,Medium,Medium,Medium,Medium,High,High,High,High,Medium,Medium,Medium,Medium
764861,Y1A7A3,17,15,24,22,20,39,35,31,33,10,17,8,22,25,13,2,3,19,26,50,106784,113973,78,11,7,2,0,94,5,0,10,89,17,0,0,0,0,82,0,5,14,8,10,26,3,17,12,0,1,45,42,9,2,0,70,29,0,35,

In [64]:
len(rename.keys())

95

In [71]:
flatfilejoin = flatfilejoin.rename(columns=rename)

In [73]:
flatfilejoin.head(1)

,POSTCODE,Percentage of total population in this postalcode that belongs to people between ages 0 -14 years,Percentage of total population in this postalcode that belongs to people between ages 15 -29 years,Percentage of total population in this postalcode that belongs to people between ages 30 -44 years,Percentage of total population in this postalcode that belongs to people between ages 45 - 59 years,Percentage of total population in this postalcode that belongs to people aged 60 and above,"Median age of total population: half of the population below this age, the other half above this age.",Percentage of population above age 15 years married,Percentage of population above age 15 years never married,"Percentage of population above age 15 years others (living common law, seperated, divorced and widowed)","% percentage of population in private households with No certificate, diploma or degree",% percentage of population in private households with Highest certificate : Secondary (high) school diploma or equivalency certificate,% percentage of population in private households with Highest certificate : Apprenticeship or trades certificate or diploma,"% percentage of population in private households with Highest certificate : College, CEGEP or other non-university certificate or diploma or University certificate or diploma below bachelor level",% percentage of population in private households with Highest certificate : Bachelor's degree,% percentage of population in private households with Highest certificate : University certificate or diploma above bachelor level or Master's degree,"% percentage of population in private households with Highest certificate : Degree in medicine, dentistry, veterinary medicine or optometry or Earned doctorate","% Percentage of private households with total household income under $25,000","% Percentage of private households with total household income from $25,000 to $59,999","% Percentage of private households with total household income from $60,000 to $99,999","% Percentage of private households with total household income $100,000 and over",Median total household income ($),Average household total income ($),"% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: Car, truck, van - as a passenger or driver",% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Public transit,% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Walked,% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Bicycle,% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Other method,Total - Commuting duration for the employed labour force aged 15 years and over in private households with a usual place of work or no fixed workplace address: 0 to 29 minutes,30 to 59 minutes,60 minutes and over,Total visible minority population,Not a visible minority. Includes respondents who reported 'Yes' to Aboriginal (self-reporting) as well as respondents who were not considered to be members of a visible minority group.,Percentage of South Asian Population in Visible Minority,Percentage of Chinese Population in Visible Minority,Percentage of Black Population in Visible Minority,Percentage of Latin American Population in Visible Minority,Percentage of Arab and West Asian Population in Visible Minority,"Percentage of Filipino, Southeast Asian, Korean and Japanese Population in Visible Minority",Others,% of labour froce population aged 15 years and over with Management occupations,

# saving the final file

In [74]:
flatfilejoin.to_csv('s3://{}/{}'.format('myhomely/raja/Manifold-selected', 'flatfile' + '.csv'))

In [75]:
flatfilejoin.shape

(764863, 96)

In [76]:
# postallookup = pd.read_csv('s3://{}/{}'.format('myhomely/source_data/msd', 'PostCodesLookupTable.csv'),encoding = 'latin-1')

In [77]:
# flatfilemini = flatfile[flatfile.Postalcode]
flatfilemini = flatfilejoin[flatfilejoin.POSTCODE.str.startswith('M')]

In [78]:
flatfilemini.shape

(42353, 96)

In [79]:
flatfilemini.to_csv('s3://{}/{}'.format('myhomely/raja/Manifold-selected', 'flatfilemini' + '.csv'))

In [80]:
flatfilemini = pd.read_csv('s3://{}/{}'.format('myhomely/raja/Manifold-selected', 'flatfilemini' + '.csv')).drop(columns = ['Unnamed: 0'])

In [81]:
flatfilemini['POSTCODE'].values

array(['M1B0A1', 'M1B0A3', 'M1B0A4', ..., 'M9W7J4', 'M9W7J5', 'M9W7J6'],
      dtype=object)

In [82]:
flatfilemini.head()

,POSTCODE,Percentage of total population in this postalcode that belongs to people between ages 0 -14 years,Percentage of total population in this postalcode that belongs to people between ages 15 -29 years,Percentage of total population in this postalcode that belongs to people between ages 30 -44 years,Percentage of total population in this postalcode that belongs to people between ages 45 - 59 years,Percentage of total population in this postalcode that belongs to people aged 60 and above,"Median age of total population: half of the population below this age, the other half above this age.",Percentage of population above age 15 years married,Percentage of population above age 15 years never married,"Percentage of population above age 15 years others (living common law, seperated, divorced and widowed)","% percentage of population in private households with No certificate, diploma or degree",% percentage of population in private households with Highest certificate : Secondary (high) school diploma or equivalency certificate,% percentage of population in private households with Highest certificate : Apprenticeship or trades certificate or diploma,"% percentage of population in private households with Highest certificate : College, CEGEP or other non-university certificate or diploma or University certificate or diploma below bachelor level",% percentage of population in private households with Highest certificate : Bachelor's degree,% percentage of population in private households with Highest certificate : University certificate or diploma above bachelor level or Master's degree,"% percentage of population in private households with Highest certificate : Degree in medicine, dentistry, veterinary medicine or optometry or Earned doctorate","% Percentage of private households with total household income under $25,000","% Percentage of private households with total household income from $25,000 to $59,999","% Percentage of private households with total household income from $60,000 to $99,999","% Percentage of private households with total household income $100,000 and over",Median total household income ($),Average household total income ($),"% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: Car, truck, van - as a passenger or driver",% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Public transit,% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Walked,% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Bicycle,% of employed labor force aged 15 years and over in private households households with a usual place of work or no fixed workplace address with main mode of commute: 'Other method,Total - Commuting duration for the employed labour force aged 15 years and over in private households with a usual place of work or no fixed workplace address: 0 to 29 minutes,30 to 59 minutes,60 minutes and over,Total visible minority population,Not a visible minority. Includes respondents who reported 'Yes' to Aboriginal (self-reporting) as well as respondents who were not considered to be members of a visible minority group.,Percentage of South Asian Population in Visible Minority,Percentage of Chinese Population in Visible Minority,Percentage of Black Population in Visible Minority,Percentage of Latin American Population in Visible Minority,Percentage of Arab and West Asian Population in Visible Minority,"Percentage of Filipino, Southeast Asian, Korean and Japanese Population in Visible Minority",Others,% of labour froce population aged 15 years and over with Management occupations,

In [265]:
len(flatfilemini.mean().values.round(2))

79

# Replacing the null values

In [ ]:
gc.collect()

In [ ]:
flatfile = pd.read_csv('s3://{}/{}'.format('myhomely/raja/Manifold-selected', 'flatfile' + '.csv')).drop(columns = ['Unnamed: 0'])

In [ ]:
flatfile.head()

In [ ]:
flatfile = flatfile.replace('Not Applicable',np.nan)

In [ ]:
flatfile.shape

In [ ]:
flatfile.isna().sum()

In [ ]:
flatfile['FSA'] = flatfile['POSTCODE'].apply(lambda x: x[:3])

In [ ]:
flatfile.head()

In [ ]:
aggdict = {}
for col in flatfile.columns.tolist()[1:-1]:
    aggdict[col] = ['mean']

In [ ]:
RPS = [x for x in flatfile.columns.tolist()[1:-1] if 'RPS' in x ]

In [ ]:
RPSnumeric = [x for x in RPS if int(x[3:]) < 38]

In [ ]:
RPSnumeric

In [ ]:
flatfile.dtypes

In [ ]:
flatfile.groupby('FSA').agg(aggdict)

In [ ]:
flatfilemini = pd.read_csv('s3://{}/{}'.format('myhomely/raja/Manifold-selected', 'flatfilemini' + '.csv')).drop(columns = ['Unnamed: 0'])